In [1]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=d5de3449db3f6bc5ac968ac26fb8c2e8e884199698e8af9260b30fb6809b41fd
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 3.4 MB/s eta 0:00:0000:0100:03
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
import collections.abc
import collections
import cv2
import json
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
import pylab
import re
import shutil
import tensorflow as tf
import time as time
import wget
import zipfile
import pycocotools
#import torch

from pycocotools import mask as COCOmask
from cv2 import dnn_superres
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from PIL import Image
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from time import time

In [4]:


def create_dir(path):
  try: 
      if not os.path.exists(path):
        os.makedirs(path) 
      else:
        shutil.rmtree(path, ignore_errors=True)
        os.makedirs(path)  
  except OSError as error: 
      print ('An error occurred creating the directory in the following path: ',path,' : ',error)
        
def know_dimensions(path):
  img = cv2.imread(path)
  height, width, channels = img.shape
  print(height, width, channels)


def load_image_into_numpy_array(path):
    return np.array(Image.open(path))


def createSR(image_path):
  IMAGE_ORIGINAL_PATH = image_path
  IMAGE_SR_PATH = '/opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png'
  PATH_TO_MODEL_DIR = '/opt/share/IWINAC-EXTENDED-FOLDER/resultados/MODEL_SR/FSRCNN_x2.pb'
  sr = dnn_superres.DnnSuperResImpl_create()
  image = cv2.imread(IMAGE_ORIGINAL_PATH)
  sr.readModel(PATH_TO_MODEL_DIR)
  sr.setModel("fsrcnn", 2)
  result = sr.upsample(image)
  cv2.imwrite(IMAGE_SR_PATH, result)
  #img = cv2.imread('/opt/share/IWINAC-EXTENDED-FOLDER/resultados/0_SR.png')
  #dst = cv2.fastNlMeansDenoisingColored(img,None,8,8,7,11)
  #cv2.imwrite('/opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png', dst)
    
    
    
def download_model_SR(model_name, model_dir):
    base_url = 'https://github.com/Saafke/FSRCNN_Tensorflow/blob/master/models/'
    model_file = base_url+ model_name +'.pb?raw=true'
    download_model = model_dir
    wget.download(model_file, download_model) 
    return download_model


def download_model(model_name, model_date):
    base_url = 'http://download.tensorflow.org/models/object_detection/tf2/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(fname=model_name,
                                        origin=base_url + model_date + '/' + 
                                        model_file,
                                        untar=True)
    return str(model_dir)

def download_labels(filename):
    base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
    label_dir = tf.keras.utils.get_file(fname=filename,
                                        origin=base_url + filename,
                                        untar=False)
    label_dir = pathlib.Path(label_dir)
    return str(label_dir)

In [5]:
import tensorflow as tf
def tensorflow_2_x_dark_magic_to_restrict_memory_use(GPU_TO_USE):

    #tf.debugging.set_log_device_placement(True)
    gpus = tf.config.experimental.list_physical_devices('GPU')
    print(gpus)

    if gpus:
        # Restrict TensorFlow to only use GPU GPU_TO_USE.
        try:
            tf.config.experimental.set_visible_devices(gpus[GPU_TO_USE], 'GPU')             # Set as visible only GPU GPU_TO_USE
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)                # Set memory grow.
            #tf.config.experimental.set_virtual_device_configuration(
            #    gpus[GPU_TO_USE],
            #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)])    # Set maximum memory size on GPU_TO_USE to 10GB
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
        except RuntimeError as e:
            # Visible devices must be set before GPUs have been initialized
            print(e)
            
tensorflow_2_x_dark_magic_to_restrict_memory_use(0)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2 Physical GPUs, 1 Logical GPU


In [6]:
#PLEASE DEFINE WHERE DO YOU WANT TO CREATE THE FOLDER TO SAVE THE SR MODEL
OUTPUT_DIR = '/opt/share/IWINAC-EXTENDED-FOLDER/resultados/'
create_dir(OUTPUT_DIR)

MODEL_NAME = 'FSRCNN_x2'
MODEL_SR_DIR = OUTPUT_DIR+'MODEL_SR'
create_dir(MODEL_SR_DIR)
PATH_TO_MODEL_DIR = download_model_SR(MODEL_NAME, MODEL_SR_DIR+'/FSRCNN_x2.pb')

MODEL_DATE = '20200711'
MODEL_NAME = 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8'
PATH_TO_MODEL_DIR = download_model(MODEL_NAME, MODEL_DATE)

LABEL_FILENAME = 'mscoco_complete_label_map.pbtxt'
PATH_TO_LABELS = download_labels(LABEL_FILENAME)

import time
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
print('Loading model...', end='')
start_time = time.time()
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

100% [..........................................................] 38973 / 38973Downloading data from http://download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz
16384/5643 [=======================================================================================] - 0s 0us/step
Loading model...Done! Took 19.0964777469635 seconds


In [7]:
def Optimizer_V2(cliques_set, limit):
    print('ENTRO EN OPTIMIZER_V2')
    dict_items = {}
    cliques_list = []
    sorted_cliques = sorted(cliques_set, key=len, reverse=False)
    while len(sorted_cliques) != 0:
        clique_extracted = sorted_cliques.pop()
        cliques_list.append(clique_extracted)
        copy_cliques = sorted_cliques

        for element in clique_extracted:
            dict_items[element] = dict_items.get(element, 0) + 1
            if dict_items[element] >= limit:
                for cliques_selected in range(len(copy_cliques)):
                    copy_cliques[cliques_selected].discard(element)
                
            copy_cliques = [x for x in copy_cliques if x != set()]
            discard_cliques = []
            
            copy_cliques = sorted(copy_cliques, key=len, reverse=True)
            
            for cliques_selected_1 in range(len(copy_cliques)):
                exit_subset = False
                index = 0
                while index < len(copy_cliques) and exit_subset == False:
                   if cliques_selected_1 != index:
                       a = copy_cliques[cliques_selected_1]
                       b = copy_cliques[index]
                       if a.issubset(b):
                          exit_subset = True
                   index = index + 1
                if exit_subset == False:
                      discard_cliques.append(copy_cliques[cliques_selected_1])
                        
            sorted_cliques = discard_cliques
    print('SALGO DE OPTIMIZER_V2')
    return cliques_list

In [8]:
import numpy as np
from object_detection.utils import ops as utils_ops
from object_detection.utils import visualization_utils as viz_utils
from pycocotools import mask

class Node(object):
    
    def __init__(self, name, coordenadas, coordenadas2,puntos):
        self.name = name
        self.neighbors = []
        self.coordenadas = coordenadas
        self.coordenadas2 = coordenadas2
        self.x1x2y1y2 = puntos
        
    def __repr__(self):
        return self.name
    
def find_cliques(potential_clique=[], remaining_nodes=[], skip_nodes=[], depth=0,cliques_list=[]):
    if len(remaining_nodes) == 0 and len(skip_nodes) == 0:
        cliques_list.append(potential_clique)
        return 1
    found_cliques = 0
    for node in remaining_nodes:
        new_potential_clique = potential_clique + [node]
        new_remaining_nodes = [n for n in remaining_nodes if n in node.neighbors]
        new_skip_list = [n for n in skip_nodes if n in node.neighbors]
        found_cliques += find_cliques(new_potential_clique, new_remaining_nodes, new_skip_list, depth + 1,cliques_list)
        remaining_nodes.remove(node)
        skip_nodes.append(node)
    return found_cliques

def centroid1(points):
    x_coords = [p[0] for p in points]
    y_coords = [p[1] for p in points]
    _len = len(points)
    centroid_x = sum(x_coords)/_len
    centroid_y = sum(y_coords)/_len
    return [centroid_x, centroid_y]

def Optimizer_V2(cliques_set, limit):

    dict_items = {}
    cliques_list = []
    sorted_cliques = sorted(cliques_set, key=len, reverse=False)
    while len(sorted_cliques) != 0:
        clique_extracted = sorted_cliques.pop()
        cliques_list.append(clique_extracted)
        copy_cliques = sorted_cliques

        for element in clique_extracted:
            dict_items[element] = dict_items.get(element, 0) + 1
            if dict_items[element] >= limit:
                for cliques_selected in range(len(copy_cliques)):
                    copy_cliques[cliques_selected].discard(element)
                
            copy_cliques = [x for x in copy_cliques if x != set()]
            discard_cliques = []
            
            for cliques_selected_1 in range(len(copy_cliques)):
                exit_subset = False
                index = 0
                while index < len(copy_cliques) and exit_subset == False:
                   if cliques_selected_1 != index:
                       a = copy_cliques[cliques_selected_1]
                       b = copy_cliques[index]
                       if b.issubset(a):
                          exit_subset = True
                   index = index + 1
                if exit_subset == False:
                      discard_cliques.append(copy_cliques[cliques_selected_1])
            sorted_cliques = discard_cliques

    return cliques_list



def bron_kerbosch(adj_matrix : np.ndarray, pivot : bool = False) -> list:
    maximal_cliques = []

    def N(v):
        return {i for i, weight in enumerate(adj_matrix[v]) if weight}

    def _bron_kerbosch(R, P, X):
        if not P and not X:
            maximal_cliques.append(R)
        else:
            if pivot:
                u = max(P | X, key=lambda i: len(N(i)))
                _P = P.copy() - N(u)
            else:
                _P = P.copy()

            for v in _P:
                _bron_kerbosch(R | {v}, P & N(v), X & N(v))
                P.remove(v)
                X.add(v)

    R, P, X = set(), set(range(len(adj_matrix))), set()
    _bron_kerbosch(R, P, X)

    return maximal_cliques

def generate_extra_points_V2(path_imageRAW, path_imageSR):
    
  imagen = cv2.imread(path_imageSR)
  height, width, _ = imagen.shape

  
  left_x = int(width/4)
  right_x   = int(width/2)+int(width/4)
  up_y    = int(height/4)
  down_y     = int(height/2)+int(height/4)

  center_x = int(width/2)
  center_y = int(height/2)

  center_coordinates_arriba_izquierda = (left_x,up_y)
  center_coordinates_arriba_derecha   = (right_x,up_y)
  center_coordinates_abajo_izquierda  = (left_x,down_y)
  center_coordinates_abajo_derecha    = (right_x,down_y)
  center_coordinates_centro           = (center_x,center_y)


  imagen2 = cv2.imread(path_imageRAW)
  RECORTEY  = int(imagen2.shape[0]/2)
  RECORTEX  = int(imagen2.shape[1]/2)

  extra_coordinates = [center_coordinates_arriba_izquierda,
                        center_coordinates_arriba_derecha,
                        center_coordinates_abajo_izquierda,
                        center_coordinates_abajo_derecha,
                        center_coordinates_centro
                       ]

  result = []
  for point in extra_coordinates:
    a1 = point[0] #es la x
    a2 = point[1] #es la y
    
    result.append(((int(a1-RECORTEX)),(int(a2-RECORTEY)),(int(a1+RECORTEX)),(int(a2+RECORTEY)) ))

  return result



In [9]:
def dame_la_mascara_v4(mascara, coordenadas, coordenadas2, width,height):
    
      #COORDENADAS ES LAS COORDENADAS DE LA IMAGEN SR
      #COORDENADAS2 ES LAS COORDENADAS DE LA IMAGEN ORIGINAL
    
      ancho,largo = mascara.shape
      #print(ancho,largo)
        
      ymax, xmax, ymin, xmin = coordenadas
      ymax11, xmax11, ymin11, xmin11 = coordenadas2
    
      puntero_x = 0
      puntero_y = 0
    
      xmax = xmax + 2
      ymax = ymax + 2
      xmin = xmin - 2
      ymin = ymin - 2
    
      #print(width, height)
    
      if xmax > width:
        xmax = width
        
      if ymax > height:
        ymax = height
        
      if xmin < 0:
        xmin = 0
        
      if ymin < 0:
        ymin = 0
        
        
      xmax11 = xmax11 + 2
      ymax11 = ymax11 + 2
      xmin11 = xmin11 - 2
      ymin11 = ymin11 - 2
    
      if xmax11 >= width:
        xmax11 = width - 1
        
      if ymax11 >= height:
        ymax11 = height - 1
        
      if xmin11 < 0:
        xmin11 = 0
        
      if ymin11 < 0:
        ymin11 = 0
    
    
      #CREO UNA MASCARA DEL TAMAÑO DE LA BBOX
      mascara_2 = np.zeros((int(ymax-ymin),int(xmax-xmin)),dtype=np.uint8)
    
    
      
      mascara2_y = 0
      for start_in_x in range(int(ymin),int(ymax)):
            mascara2_x = 0
            for start_in_y in range(int(xmin),int(xmax)):
                mascara_2[mascara2_y][mascara2_x] = mascara[start_in_x][start_in_y]
                mascara2_x = mascara2_x + 1
            mascara2_y = mascara2_y + 1
      
     
    
      #HAGO EL RESIZE DE LA MASK
      mascara_correcta1 = mascara_2*255
      ancho2,largo2 = mascara_correcta1.shape
      mascara_correcta2 = cv2.resize(mascara_correcta1, (int(largo2/2),int(ancho2/2)), interpolation=cv2.INTER_CUBIC)
      mascara_correcta22 = np.where(mascara_correcta2 > 128, 1, 0)
      mascara_correcta22 = mascara_correcta22.astype(np.uint8)
                    
      #HAGO LA TRASLACIÓN DE LAS COORDENADAS   
      mascara_correcta3 = mascara_correcta22
      mascara_auxiliar = np.zeros((height,width),dtype=np.uint8)
    
      inicioyminimo = int(ymin11)
      inicioymaximo = int(ymax11)
      inicioxminimo = int(xmin11)
      inicioxmaximo = int(xmax11)
    
      
      recorredor_x = 0
      recorredor_y = 0
    
      start_y = inicioyminimo
      start_x = inicioxminimo
    
    
      start_x = inicioyminimo
      
      
      for row in mascara_correcta22:
        start_y = inicioxminimo
        for val in row:
            try:
                mascara_auxiliar[start_x][start_y] = val
                start_y = start_y + 1
            except:
                pass
               
        start_x = start_x + 1
     
      
    
      #DEVUELVO LA MASCARA YA PROCESADA  
      return mascara_auxiliar




def symmetric_pad_array(input_array: np.ndarray, target_shape: tuple, pad_value: int) -> np.ndarray:

    for dim_in, dim_target in zip(input_array.shape, target_shape):
        if dim_target < dim_in:
            raise Exception("`target_shape` should be greater or equal than `input_array` shape for each axis.")

    pad_width = []
    for dim_in, dim_target  in zip(input_array.shape, target_shape):
        if (dim_in-dim_target)%2 == 0:
            pad_width.append((int(abs((dim_in-dim_target)/2)), int(abs((dim_in-dim_target)/2))))
        else:
            pad_width.append((int(abs((dim_in-dim_target)/2)), (int(abs((dim_in-dim_target)/2))+1)))
    
    return np.pad(input_array, pad_width, 'constant', constant_values=pad_value)




def dame_la_mascara_v5(mascara, coordenadas, coordenadas2, width,height):
    
      #COORDENADAS ES LAS COORDENADAS DE LA IMAGEN SR
      #COORDENADAS2 ES LAS COORDENADAS DE LA IMAGEN ORIGINAL
    
      ancho,largo = mascara.shape
      #print(ancho,largo)
        
      ymax, xmax, ymin, xmin = coordenadas
      ymax11, xmax11, ymin11, xmin11 = coordenadas2
    
      puntero_x = 0
      puntero_y = 0
    
      
    
      #CREO UNA MASCARA DEL TAMAÑO DE LA BBOX
      mascara_2 = np.zeros((int(ymax-ymin),int(xmax-xmin)),dtype=np.uint8)
      mascara_2 = mascara[ymin:ymax, xmin:xmax]
    
      
      #HAGO EL RESIZE DE LA MASK
      mascara_correcta1 = mascara_2*255
      ancho2,largo2 = mascara_correcta1.shape
      mascara_correcta2 = cv2.resize(mascara_correcta1, (int(largo2/2),int(ancho2/2)), interpolation=cv2.INTER_CUBIC)
      mascara_correcta22 = np.where(mascara_correcta2 > 128, 1, 0)
      mascara_correcta22 = mascara_correcta22.astype(np.uint8)
                    
      #HAGO LA TRASLACIÓN DE LAS COORDENADAS   
      mascara_correcta3 = mascara_correcta22
      mascara_auxiliar = np.zeros((height,width),dtype=np.uint8)
    
      inicioyminimo = int(ymin11)
      inicioymaximo = int(ymax11)
      inicioxminimo = int(xmin11)
      inicioxmaximo = int(xmax11)
    
      print(ymax11-ymin11,xmax11-xmin11,mascara_correcta22.shape)
    
      mascara_correcta23 = symmetric_pad_array(mascara_correcta22, (inicioymaximo-inicioyminimo,inicioxmaximo-inicioxminimo,3), pad_value=0)
    
      inicioyminimo = int(ymin11)
      inicioymaximo = int(ymax11)
      inicioxminimo = int(xmin11)
      inicioxmaximo = int(xmax11)
    
      
    
      mascara_auxiliar[inicioyminimo:inicioymaximo, inicioxminimo:inicioxmaximo] = mascara_correcta23

    
      #DEVUELVO LA MASCARA YA PROCESADA  
      return mascara_auxiliar

In [10]:
"""
Author: Jorge García <sirsykon@gmail.com>
"""
"""
File to create test time augmentation functions to transform images in order to be applied to be processed by an object detection method.

The idea is to return a function to transform the images and a function to transform each object detected to the original position so we can get the detection on the original image coordinates.

All functions will assume mathematical point standard (width, height).

"""
import numpy as np

def intersection_between_two_masks(mask1, mask2):
    """
    Function to calculate the intersection area between to masks. Masks should have the same shape and are assumed to be boolean or binary.

    Args:
        mask1 ([type]): [description]
        mask2 ([type]): [description]

    Returns:
        [type]: [description]
    """
    
    print(mask1.shape,mask2.shape)
    assert mask1.shape==mask2.shape

    intersection_mask = np.logical_and(mask1, mask2)*1

    return np.sum(intersection_mask)

def union_between_two_masks(mask1, mask2):
    """
    Function to calculate the union area between to masks. Masks should have the same shape and are assumed to be boolean or binary.

    Args:
        mask1 ([type]): [description]
        mask2 ([type]): [description]

    Returns:
        [type]: [description]
    """
    assert mask1.shape==mask2.shape
    
    union_mask = np.logical_or(mask1, mask2)*1

    return np.sum(union_mask)
 

def intersection_over_union_using_masks(mask1, mask2):

    intersection = intersection_between_two_masks(mask1, mask2)
    union = union_between_two_masks(mask1, mask2)

    return intersection/union

def compute_overlaps_masks2(masks1, masks2):
    
    a1 = np.count_nonzero(masks1 == 1)
    a2 = np.count_nonzero(masks2 == 1)
    
    new_mask = masks1+masks2
    a3 = np.count_nonzero(new_mask == 2)
    
    percentage1 = a3*100/a1
    percentage2 = a3*100/a2
    
    if percentage2 > percentage1:
        percentage1= percentage2
    
    return percentage1/100


def get_most_confiable_object_idx(lists):
    max_confidence = None
    max_confidence_object_idx = None
    max_confidence_object_list_idx = None
    
    
    #INICIALIZAMOS EL SCORE MAYOR A NONE, ASÍ COMO EL IDENTIFICADOR DEL OBJETO Y DE LA LISTA
    for l_idx, l in enumerate(lists):
        _, _, confidences = l
        
        #SELECCIONAMOS EL IDENTIFICADOR EN LA POSICIÓN DONDE SE ENCUENTRE EL MAYOR SCORE
        current_list_max_confidence = confidences.max()
        
        #REALIZAMOS LA COMPARACIÓN PREVIA PARA DETERMINAR QUE SEA MAYOR QUE EL QUE TENEMOS ALMACENADO PREVIAMENTE
        if not max_confidence or max_confidence < current_list_max_confidence and l_idx==0:
            max_confidence = current_list_max_confidence
            max_confidence_object_idx = np.where(confidences==max_confidence)[0][0]
            max_confidence_object_list_idx = l_idx
            
    #DEVOLVEMOS UNA VEZ RECORRIDAS TODAS LAS LISTAS DISPONIBLES EL ID DEL OBJETO CON MAYOR PUNTUACIÓN Y EL ID DE LA
    #LISTA QUE CONTIENE AL MISMO
    return max_confidence_object_idx, max_confidence_object_list_idx






def get_most_confiable_object_idx_2(lists):
    max_confidence = None
    max_confidence_object_idx = None
    max_confidence_object_list_idx = None
    
    
    #INICIALIZAMOS EL SCORE MAYOR A NONE, ASÍ COMO EL IDENTIFICADOR DEL OBJETO Y DE LA LISTA
    for l_idx, l in enumerate(lists):
        masks_print , classes_print , confidences = l
        
        
        lista2 = []
        for element in masks_print:
            lista2.append(np.count_nonzero(element == 1))
            
        lista2 = np.array(lista2)
        
        #SELECCIONAMOS EL IDENTIFICADOR EN LA POSICIÓN DONDE SE ENCUENTRE EL MAYOR SCORE
        current_list_max_confidence = lista2.max()
        
        #REALIZAMOS LA COMPARACIÓN PREVIA PARA DETERMINAR QUE SEA MAYOR QUE EL QUE TENEMOS ALMACENADO PREVIAMENTE
        if not max_confidence or max_confidence < current_list_max_confidence:
            max_confidence = current_list_max_confidence
            max_confidence_object_idx = np.where(lista2==max_confidence)[0][0]
            max_confidence_object_list_idx = l_idx
            
    #DEVOLVEMOS UNA VEZ RECORRIDAS TODAS LAS LISTAS DISPONIBLES EL ID DEL OBJETO CON MAYOR PUNTUACIÓN Y EL ID DE LA
    #LISTA QUE CONTIENE AL MISMO
    return max_confidence_object_idx, max_confidence_object_list_idx




def get_same_object(obj, detections_list, thd=0.7, ignore_class = True):
    """Function to get the nearest object to obj from list of object with minimum an IOU of thd.

    Args:
        obj (tuple(bbox, class, confidence)): obj to compare.
        detections_list (tuple(list(bbox), list(class), list(confidence))): List containing dected objects to compare obj.
        thd (float): Minimum iou to be a candidate.
        ignore_class (bool, optional): Do we ignore class to assign equivalence?. Defaults to True.

    Returns:
        int: idx from detections_list[0] with the greater iou compared to obj.
    """
    
    detections_list2 =  tuple(list(detections_list))
    list_mask_copy, list_classes_copy, list_confidences_copy = detections_list2
    
    thd = 0.5
    
    ignore_class = True
    list_idx_return = []

    #OBTENGO INFORMACIÓN DEL ELEMENTO CON EL CUAL REALIZAR LA CORRELACIÓN
    list_masks, list_classes, list_confidencess = detections_list
    obj_mask, obj_class, _ = obj
    
    greater_iou = None
    greater_iou_idx = None
    current_list_processed_v2 = None
    
    print('STARTTT ',len(list_masks))
    #CALCULO EL IoU PARA TODOS LOS ELEMENTOS DE DICHA LISTA
    for idx, (mask, _class) in enumerate(zip(list_masks, list_classes)):
        #print('tamanio ',len(list_masks))
        #CALCULO EL IoU EN LOS DOS SENTIDOS PARA DESCARTAR PEQUEÑAS PORCIONES DE UN MISMO OBJETO
        iou1 = compute_overlaps_masks2(mask, obj_mask)
        
        #thd = 0.05
        #iou1 = intersection_over_union_using_masks(mask,obj_mask)
        
        if ignore_class or obj_class == _class:

            #SI EL VALOR DEL IoU CALCULADO SUPERA UN CIERTO UMBRAL, ALMACENO LA POSICION EN LA QUE SE ENCUENTRA
            print(iou1)
            if iou1 >= thd:
                list_idx_return.append(idx)
                
                
    returna = []
    returnb = []
    returnc = []
    #print('TENGO QUE ELIMINAR ',list_idx_return)
    
    for elementalisimo_idx in range(len(list_masks)):
        #print('COMPROBACION ',elementalisimo_idx,list_idx_return)
        if elementalisimo_idx not in list_idx_return:
            print(elementalisimo_idx, ' NO ESTA EN ',list_idx_return)
            returna.append(list_masks[elementalisimo_idx])
            returnb.append(list_classes[elementalisimo_idx])
            returnc.append(list_confidencess[elementalisimo_idx])
    #print('LO QUE DEVUELVO ',len(returna))
    print('ENDDDD ',len(list_idx_return))
    #DEVUELVO LA LISTA CON TODOS LOS IDENTIFICADORES
    return (list_idx_return, (returna,np.array(returnb),np.array(returnc)))



def create_clusters(_lists, threshold):
    """Function to create clusters by assigning equivalences between N>1 lists of objects based on their bboxes IoU.

    Args:  
        _lists (list(tuple)) : An arbitrary list of tuples with structure (np.array(bbox), np.array(classs), np.array(confidences))

    Returns:

    """
    
    
    
    
    #EN PRIMER LUGAR SE HACE UNA COPIA DE LAS LISTAS SOBRE LAS CUALES SE DESEA ITERAR
    lists = _lists.copy()
    
   
    
    #ES POSIBLE QUE ALGUNA DE DICHAS LISTAS ESTEN VACÍAS, ES POR ELLO POR LO QUE HAY QUE ELIMINARLAS PREVIAMENTE
    empty_lists_to_pop_idx = []
    for idx, _list in enumerate(lists):
        _, classes, _ = _list
        if classes.shape[0] ==0:
            empty_lists_to_pop_idx.append(idx)

    #UNA VEZ QUE CONOCEMOS LOS IDS DE LAS LISTAS VACIAS, PROCEDEMOS A ELIMINARLAS
    copy_list_local = lists
    new_list_local = []
    
    for element_list in range(len(copy_list_local)):
        if element_list not in empty_lists_to_pop_idx:
            new_list_local.append(lists[element_list])
        
    lists = new_list_local
    
   

    #GENERAMOS EL CLUSTER VACÍO EL CUAL VAMOS A DEVOLVER CUANDO FINALICE EL ALGORITMO
    clusters = []  
    
    #MIENTRAS EXISTAN LISTAS QUE PROCESAR, SE CONTINUA CON EL ALGORITMO
    while len(lists) > 0:
        
        
        #OBTENEMOS EL OBJETO CON MAYOR PUNTUACIÓN DISPONIBLE DE LAS LISTAS PENDIENTES DE PROCESAR
        most_confiable_object_idx, most_confiable_object_list_idx = get_most_confiable_object_idx_2(lists)
        
        #OBTENEMOS POR TANTO EN EL PASO ANTERIOR EL ID DEL OBJETO, ASÍ COMO EL ID DE LA LISTA QUE LO CONTIENE
        most_confiable_object_list = lists[most_confiable_object_list_idx]
        most_confiable_object_list_masks, most_confiable_object_list_classes, most_confiable_object_list_confidences = most_confiable_object_list

        #GENERAMOS LA TUPLA A PARTIR DEL OBJETO SELECCIONADO
        most_confiable_object_mask       = most_confiable_object_list_masks[most_confiable_object_idx]
        most_confiable_object_class      = most_confiable_object_list_classes[most_confiable_object_idx]
        most_confiable_object_confidence = most_confiable_object_list_confidences[most_confiable_object_idx]
        
        #YA TENGO EL PRIMER ELEMENTO ESTABLECIDO EN EL CLUSTER
        cluster_bboxes = [most_confiable_object_mask]
        cluster_classes = [most_confiable_object_class]
        cluster_confidences = [most_confiable_object_confidence]
        print('HE METIDO UNOOOOOO')
        
        #AHORA HAY QUE ELIMINAR EL MISMO DE LA LISTA DEL CUAL HA SIDO SELECCIONADO
        most_confiable_object_list_masks.pop(most_confiable_object_idx)
        most_confiable_object_list_classes = np.delete(most_confiable_object_list_classes, most_confiable_object_idx)
        most_confiable_object_list_confidences = np.delete(most_confiable_object_list_confidences, most_confiable_object_idx)
        
        
        if len(most_confiable_object_list_masks) == 0:
            lists.pop(most_confiable_object_list_idx)
        else:
            new_tuple_generated = (most_confiable_object_list_masks,most_confiable_object_list_classes,most_confiable_object_list_confidences)
            lists[most_confiable_object_list_idx] = new_tuple_generated
            
        #POSTERIORMENTE, HAY QUE BUSCAR COINCIDENCIA DE DICHO OBJETO EN EL RESTO DE LISTAS
        most_confiable_object = (most_confiable_object_mask, most_confiable_object_class, most_confiable_object_confidence)
    
        list_to_delete = []
        
        element = 0
        indexador = []
        while element < len(lists):
        
            list_idx=element
            
            #NO EXCLUYO LA LISTA DE DONDE SAQUÉ EL ELEMENTO POR SI HA HABIDO UNA DETECCIÓN ANÓMALA
            current_list = lists[list_idx]
            
            #DEVUELVO LOS INDICES DE AQUELLOS ELEMENTOS QUE HAYAN SUPERADO UN UMBRAL DE IoU
            same_objects_from_current_list_idx, current_list_v2 = get_same_object(most_confiable_object, current_list, threshold)
            
            #SI LA LISTA QUE HE DEVUELTO NO ESTÁ VACÍA, POR CADA ID LOS AÑADO EN EL CLUSTER Y LO ELIMINO
            #POSTERIORMENTE DE DICHA LISTA
            print('ESTAS SON LAS COINCIDENCIAS ',len(same_objects_from_current_list_idx))
            if len(same_objects_from_current_list_idx) > 0:
                
                #OBTENGO LA INFORMACIÓN DE LA LISTA QUE ESTOY PROCESANDO ACTUALMENTE
                current_list_masks_selected, current_list_classes_selected, current_list_conficendes_selected = current_list
                
                for element_idx_selected_iou in same_objects_from_current_list_idx:
                    
                    #OBTENGO LA INFORMACION DEL ELEMENTO Y LA INCLUYO EN EL CLUSTER
                    cluster_bboxes.append(current_list_masks_selected[element_idx_selected_iou])
                    cluster_classes.append(current_list_classes_selected[element_idx_selected_iou])
                    cluster_confidences.append(current_list_conficendes_selected[element_idx_selected_iou])
                    
    
        
            #ELIMINO LOS ELEMENTOS QUE HAN SIDO PROCESADOS
            current_list = current_list_v2

            #assert current_list[0].shape[0] == current_list_classes.shape[0] == current_list_conficendes.shape[0]

            #SI LA LISTA QUE ESTOY PROCESANDO NO ESTÁ VACÍA, SE ACTUALIZA, EN CASO CONTRARIO, SE ELIMINA
            print('AVERRR ',len(list(current_list[0])))
            if len(list(current_list[0])) > 0:
                #print('A ',current_list)
                lists[list_idx] = current_list
            else:
                #print('b ',current_list)
                indexador.append(list_idx)
                #lists.pop(list_idx)
                
            element = element + 1
            print('HAY ',len(lists),' LISTAS')
       
    
        list_copy3 = []
        for elemental3 in range(len(lists)):
            if elemental3 not in indexador:
                list_copy3.append(lists[elemental3])
        
        lists = list_copy3
    
    
        #UNA VEZ RECORRIDA TODAS LAS LISTAS, GENERO EL RESULTADO FINAL PARA DICHA POSICIÓN
        cluster = (cluster_bboxes, np.array(cluster_classes), np.array(cluster_confidences))
        print('METO ',len(cluster_bboxes))
        clusters.append(cluster)

    return clusters



In [11]:
def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

def make_simple_inference240(image,counter,x1,y1,diccionario_final_predicciones,
                                               diccionario_final_scores,
                                               diccionario_final_clases,
                                               diciconario_final_mascaras,
                                               width,height,auxiliar_tiempos, LISTA_TRIPLAS,path_test,scorethresh,classeslist,image_path2):
  #print('Running super-inference for {}... '.format(image), end='\n')
  #print('ATENCION SCOREEEEEE ',scorethresh)
  np.set_printoptions(threshold=np.inf)
  dpi=300
  model = detect_fn
  image_np = np.array(image)
    
    
  image_np_out = load_image_into_numpy_array(image_path2)
  image2 = image_np
  image = image_np
  #image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis, ...]

  model_fn = model.signatures['serving_default']
  detections = model_fn(input_tensor)
  num_detections = int(detections.pop('num_detections'))
  need_detection_key = ['detection_classes','detection_boxes','detection_masks','detection_scores']
  detections = {key: detections[key][0, :num_detections].numpy()
                for key in need_detection_key}
  detections['num_detections'] = num_detections
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
  if 'detection_masks' in detections:
      detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
          tf.convert_to_tensor(detections['detection_masks']), detections['detection_boxes'],
          image.shape[0], image.shape[1])
      detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                        tf.uint8)
      detections['detection_masks_reframed'] = detection_masks_reframed.numpy()
  image_np_with_detections = image_np.copy()

  boxes_filter = []
  scores_filter = []
  classes_filter = []
  mask_filter = []

  for item in range(len(detections['detection_boxes'])):
        if detections['detection_classes'][item] in classeslist and detections['detection_scores'][item]>scorethresh:
            boxes_filter.append(detections['detection_boxes'][item].tolist())
            scores_filter.append(detections['detection_scores'][item])
            classes_filter.append(detections['detection_classes'][item])
            mask_filter.append(detections['detection_masks_reframed'][item])
            
  #boxes_filter, classes_filter, scores_filter, mask_filter = filterdual(boxes_filter, classes_filter, scores_filter, mask_filter)
   
  detections['detection_boxes'] = np.array(boxes_filter)
  detections['detection_classes'] = np.array(classes_filter)
  detections['detection_scores'] = np.array(scores_filter)
  detections['detection_masks_reframed'] = np.array(mask_filter)
    
  true_boxes = detections['detection_boxes']
  true_clases = detections['detection_classes']
  true_scores = detections['detection_scores']
  true_mask = detections['detection_masks_reframed']
  
  '''
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image2,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      instance_masks=detections.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=1)
  plt.axis('off')
  plt.imshow(image2)
  '''
  
  
  min_score_thresh=scorethresh

  
  nombre = counter
  
  dpi = 300
  '''
  plt.savefig(path_test+str(counter),  dpi=dpi ,bbox_inches='tight',pad_inches = 0)
  
  plt.clf()
  '''
  

  
  
  
  LISTA_BOXES_TRIPLAS = []
  LISTA_CLASES_TRIPLAS = []
  LISTA_SCORES_TRIPLAS = []
  LISTA_MASCARAS_TRIPLA = []

  for i in range(len(true_boxes)):
    box_detected = true_boxes[i]
    ymin = int(box_detected[0]*height)
    xmin = int(box_detected[1]*width)
    ymax = int(box_detected[2]*height)
    xmax = int(box_detected[3]*width) 
    pointax = ((xmax)-(xmin))/2

    ymin11 = ((((box_detected[0]*height))+y1)/2)
    xmin11 = ((((box_detected[1]*width))+x1)/2)
    ymax11 = ((((box_detected[2]*height))+y1)/2)
    xmax11 = ((((box_detected[3]*width))+x1)/2)

    coordenadas_good = []
    coordenadas_good.append(ymin11/height)
    coordenadas_good.append(xmin11/width)
    coordenadas_good.append(ymax11/height)
    coordenadas_good.append(xmax11/width)

    diccionario_final_predicciones.append(coordenadas_good)
    diccionario_final_scores.append(true_scores[i])
    diccionario_final_clases.append(true_clases[i])

    mascara = true_mask[i]

    mascara_auxiliar = dame_la_mascara_v4(mascara, [ymax, xmax, ymin, xmin], [ymax11, xmax11, ymin11, xmin11],width,height)
    mascara_auxiliar = mascara_auxiliar.astype(np.uint8)
    diciconario_final_mascaras.append(mascara_auxiliar)
    
    if np.count_nonzero(mascara_auxiliar == 1) > 0:


        LISTA_BOXES_TRIPLAS.append(coordenadas_good)
        LISTA_CLASES_TRIPLAS.append(true_clases[i])
        LISTA_SCORES_TRIPLAS.append(true_scores[i])
        LISTA_MASCARAS_TRIPLA.append(mascara_auxiliar)
    
    
    
  detections['detection_boxes'] = np.array(LISTA_BOXES_TRIPLAS)
  detections['detection_classes'] = np.array(LISTA_CLASES_TRIPLAS)
  detections['detection_scores'] = np.array(LISTA_SCORES_TRIPLAS)
  detections['detection_masks_reframed'] = np.array(LISTA_MASCARAS_TRIPLA)
  '''
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_out,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      instance_masks=detections.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=1)
  plt.axis('off')
  plt.imshow(image_np_out)
  '''
  
  
  min_score_thresh=scorethresh

  
  nombre = 'new'+str(counter)
  
  dpi = 300
  '''
  plt.savefig(path_test+str(nombre),  dpi=dpi ,bbox_inches='tight',pad_inches = 0)
  
  plt.clf()
  '''
 
  
  if len(LISTA_MASCARAS_TRIPLA) > 0:
        LISTA_TRIPLAS.append((LISTA_MASCARAS_TRIPLA, np.array(LISTA_CLASES_TRIPLAS), np.array(LISTA_SCORES_TRIPLAS)))
    
    
  
def make_inference240(image_path,counter,image_save,contador_veces,mascarathr,window,scorethresh,classeslist):
  print('Running inference for {}... '.format(image_path), end='\n')
  createSR(image_path)
    
  image_np = (image_path)
  image = image_path
    
  LISTA_TRIPLAS = []

  Nodes_Filter = []
  pather_raw = image_path

  create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/IWINAC-MASK/SIMPLE/')

    
  np.set_printoptions(threshold=np.inf)
  dpi=300
  model = detect_fn
  image_np = load_image_into_numpy_array(image)
  image2 = np.array(Image.open(image))
  image = np.array(Image.open(image))
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis, ...]

  model_fn = model.signatures['serving_default']
  detections = model_fn(input_tensor)
  num_detections = int(detections.pop('num_detections'))
  need_detection_key = ['detection_classes','detection_boxes','detection_masks','detection_scores']
  detections = {key: detections[key][0, :num_detections].numpy()
                for key in need_detection_key}
  detections['num_detections'] = num_detections
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
  if 'detection_masks' in detections:
      detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
          tf.convert_to_tensor(detections['detection_masks']), detections['detection_boxes'],
          image.shape[0], image.shape[1])
      detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                        tf.uint8)
      detections['detection_masks_reframed'] = detection_masks_reframed.numpy()
  image_np_with_detections = image_np.copy()




  min_score_thresh=.2
  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']
  mask_detected = detections['detection_masks_reframed']
    
    

  boxes_filter = []
  scores_filter = []
  classes_filter = []
  mask_filter = []

  for item in range(len(detections['detection_boxes'])):
        if detections['detection_classes'][item] in classeslist:
            boxes_filter.append(detections['detection_boxes'][item].tolist())
            scores_filter.append(detections['detection_scores'][item])
            classes_filter.append(detections['detection_classes'][item])
            mask_filter.append(detections['detection_masks_reframed'][item])
            
  #boxes_filter, classes_filter, scores_filter, mask_filter = filterdual(boxes_filter, classes_filter, scores_filter, mask_filter)
   
    
  boxes  = np.array(boxes_filter)
  clases_detected = np.array(classes_filter)
  scores = np.array(scores_filter)
  mask_detected = np.array(mask_filter)
    
    
  true_boxes  = boxes[scores > min_score_thresh]
  true_scores = scores[scores > min_score_thresh]
  true_clases = clases_detected[scores > min_score_thresh]
  true_mask = mask_detected[scores > min_score_thresh]
















  




























  
  
    
  sin_propuesta_2 = len(true_boxes)
  sin_propuesta_225 = len(true_boxes)




  LISTA_BOXES_TRIPLAS = []
  LISTA_CLASES_TRIPLAS = []
  LISTA_SCORES_TRIPLAS = []
  LISTA_MASCARAS_TRIPLA = []
  contador_triplas = 0
    
  new_counter = 0
  img = cv2.imread(image_path)
  im = Image.open('/opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png')

  contador_final = 0 

  img__READ = cv2.imread(image_path)
  height, width, channels = img__READ.shape

  diccionario_final_predicciones = []
  diccionario_final_scores = []
  diccionario_final_clases = []
  diciconario_final_mascaras = []
    
  contador_veces = contador_veces+len(true_boxes)
  array_veces.append(contador_veces)
    
  sin_propuesta = 0




  for i in range(len(true_boxes)):
    if true_scores[contador_triplas] > scorethresh:
      box_detected = true_boxes[i]
      ymin = int(box_detected[0]*height*2)
      xmin = int(box_detected[1]*width*2)
      ymax = int(box_detected[2]*height*2)
      xmax = int(box_detected[3]*width*2)
    
      ymin11 = (box_detected[0])
      xmin11 = (box_detected[1])
      ymax11 = (box_detected[2])
      xmax11 = (box_detected[3])

      coordenadas_good = []
      coordenadas_good.append(ymin11)
      coordenadas_good.append(xmin11)
      coordenadas_good.append(ymax11)
      coordenadas_good.append(xmax11)
        
      LISTA_BOXES_TRIPLAS.append(coordenadas_good)
      LISTA_CLASES_TRIPLAS.append(true_clases[contador_triplas])
      LISTA_SCORES_TRIPLAS.append(true_scores[contador_triplas])
    
      ancho2,largo2 = true_mask[contador_triplas].shape
      
      
    
    
      LISTA_MASCARAS_TRIPLA.append(true_mask[contador_triplas])
        
      contador_triplas = contador_triplas + 1
      
  
  '''
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image2,
      np.array(LISTA_BOXES_TRIPLAS),
      np.array(LISTA_CLASES_TRIPLAS),
      np.array(LISTA_SCORES_TRIPLAS),
      category_index,
      instance_masks=np.array(LISTA_MASCARAS_TRIPLA),
      use_normalized_coordinates=True,
      max_boxes_to_draw=300,
      min_score_thresh=scorethresh,
      agnostic_mode=False,
      line_thickness=1)
  '''

  '''
  display(Image.fromarray(image2))
  '''
  '''
  plt.axis('off')
  plt.imshow(image2)
  '''
  dpi = 300
  '''
  plt.savefig('/opt/share/IWINAC-EXTENDED-FOLDER/experimentos/AUXILIAR-FIX-TEST/test/FILES/ORIGINAL',  dpi=dpi ,bbox_inches='tight',pad_inches = 0)
  plt.clf()
  '''


  
  LISTA_TRIPLAS.append((LISTA_MASCARAS_TRIPLA, np.array(LISTA_CLASES_TRIPLAS), np.array(LISTA_SCORES_TRIPLAS)))
  
  #LISTA_TRIPLAS.append((np.array(LISTA_BOXES_TRIPLAS),np.array(LISTA_CLASES_TRIPLAS),np.array(LISTA_SCORES_TRIPLAS),np.array(LISTA_MASCARAS_TRIPLA)))
  #LISTA_TRIPLAS.append((np.array(LISTA_BOXES_TRIPLAS),np.array(LISTA_CLASES_TRIPLAS),np.array(LISTA_SCORES_TRIPLAS),np.array(LISTA_MASCARAS_TRIPLA)))
  #LISTA_TRIPLAS.append((np.array(LISTA_BOXES_TRIPLAS),np.array(LISTA_CLASES_TRIPLAS),np.array(LISTA_SCORES_TRIPLAS),np.array(LISTA_MASCARAS_TRIPLA)))
  #LISTA_TRIPLAS.append((np.array(LISTA_BOXES_TRIPLAS),np.array(LISTA_CLASES_TRIPLAS),np.array(LISTA_SCORES_TRIPLAS),np.array(LISTA_MASCARAS_TRIPLA)))
  
  #print(len(LISTA_TRIPLAS))
    
    
    
    
  for i in range(len(true_boxes)):
      box_detected = true_boxes[i]
      
      ymin = int(box_detected[0]*height*2)
      xmin = int(box_detected[1]*width*2)
      ymax = int(box_detected[2]*height*2)
      xmax = int(box_detected[3]*width*2)
    
      ymin11 = (box_detected[0])
      xmin11 = (box_detected[1])
      ymax11 = (box_detected[2])
      xmax11 = (box_detected[3])

      coordenadas_good = []
      coordenadas_good.append(ymin11)
      coordenadas_good.append(xmin11)
      coordenadas_good.append(ymax11)
      coordenadas_good.append(xmax11)
    
      coordenada1_y = ((box_detected[0]+box_detected[2])/2)*height
      coordenada2_x = ((box_detected[1]+box_detected[3])/2)*width
    
      Nodes_Filter.append(Node(str(i),(coordenada2_x,coordenada1_y),(int((xmin+xmax)/2),int((ymin+ymax)/2)),(xmin11*width,xmax11*width,ymin11*height,ymax11*height)))
        
  
  new_width_22 = width*window
  new_height_22 = height*window
  adj_matrix2 = []
  adj_matrix_SUPLE = []
  all_nodes = []
    
    
  '''
  #comprobacion de los centroides
  image_centered = cv2.imread(image_path)
  for centro_element in range(len(Nodes_Filter)):
        x_centered = Nodes_Filter[centro_element].coordenadas[0]
        y_centered = Nodes_Filter[centro_element].coordenadas[1]
        print(x_centered,y_centered)
        image_centered = cv2.circle(image_centered, (int(x_centered),int(y_centered)), radius=5, color=(255, 0, 0), thickness=2)
  cv2.imwrite('/opt/share/IWINAC-EXTENDED-FOLDER/testing/CENTERED.png',image_centered)
  '''

  '''
  #comprobacion de los centroides
  image_centered = cv2.imread('/opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png')
  test_centroid = []
  contadoree = 0
  for centro_element in range(len(Nodes_Filter)):
        if contadoree<4:
            x_centered = Nodes_Filter[centro_element].coordenadas2[0]
            y_centered = Nodes_Filter[centro_element].coordenadas2[1]
            test_centroid.append((x_centered,y_centered))
            image_centered = cv2.circle(image_centered, (int(x_centered),int(y_centered)), radius=30, color=(255, 0, 0), thickness=-1)
            contadoree = contadoree + 1
  
  centroide = centroid1(test_centroid)
  image_centered = cv2.circle(image_centered, (int(centroide[0]),int(centroide[1])), radius=30, color=(0, 0, 255), thickness=-1)
  
    
    
  test_centroid = [(500,500),(900,500),(500,900),(900,900)]
  centroide = centroid1(test_centroid)
  image_centered = cv2.circle(image_centered, (int(centroide[0]),int(centroide[1])), radius=30, color=(255,252 ,210), thickness=-1)
  
  image_centered = cv2.circle(image_centered, (int(500),int(500)), radius=30, color=(255,252 ,51), thickness=-1)
  image_centered = cv2.circle(image_centered, (int(500),int(900)), radius=30, color=(255,252 ,51), thickness=-1)
  image_centered = cv2.circle(image_centered, (int(900),int(500)), radius=30, color=(255,252 ,51), thickness=-1)
  image_centered = cv2.circle(image_centered, (int(900),int(900)), radius=30, color=(255,252 ,51), thickness=-1)
  
  cv2.imwrite('/opt/share/IWINAC-EXTENDED-FOLDER/testing/CENTERED.png',image_centered)
  '''
  
  

  
  


  '''
  #comprobacion de los bbox
  image_centered = cv2.imread(image_path)
  for centro_element in range(len(Nodes_Filter)):
        x_centered_min = Nodes_Filter[centro_element].x1x2y1y2[0]
        y_centered_min = Nodes_Filter[centro_element].x1x2y1y2[2]
        x_centered_max = Nodes_Filter[centro_element].x1x2y1y2[1]
        y_centered_max = Nodes_Filter[centro_element].x1x2y1y2[3]
        image_centered = cv2.circle(image_centered, (int(x_centered_min),int(y_centered_min)), radius=5, color=(255, 0, 0), thickness=2)
        image_centered = cv2.circle(image_centered, (int(x_centered_min),int(y_centered_max)), radius=5, color=(255, 0, 0), thickness=2)
        image_centered = cv2.circle(image_centered, (int(x_centered_max),int(y_centered_min)), radius=5, color=(255, 0, 0), thickness=2)
        image_centered = cv2.circle(image_centered, (int(x_centered_max),int(y_centered_max)), radius=5, color=(255, 0, 0), thickness=2)
  cv2.imwrite('/opt/share/IWINAC-EXTENDED-FOLDER/testing/CENTERED.png',image_centered)
  '''
        
    




  for i in range(len(Nodes_Filter)):

     adj_matrix_SUPLE = []
     First_Node_Selected = Nodes_Filter[i]
     lista_vecinos = []

     a1 = First_Node_Selected.coordenadas[0]
     a2 = First_Node_Selected.coordenadas[1]

     RECORTEX = int(new_width_22/2)
     RECORTEY = int(new_height_22/2)
    
     xmin = a1-RECORTEX
     ymin = a2-RECORTEY
     xmax = a1+RECORTEX
     ymax = a2+RECORTEY
      
     for o in range(len(Nodes_Filter)):
       if i != o:
         Second_Node_Selected = Nodes_Filter[o]
         
         xminb = Second_Node_Selected.x1x2y1y2[0]
         xmaxb = Second_Node_Selected.x1x2y1y2[1]
         yminb = Second_Node_Selected.x1x2y1y2[2]
         ymaxb = Second_Node_Selected.x1x2y1y2[3]
        
         if (xminb >= xmin and xmaxb <= xmax and yminb >= ymin and ymaxb <= ymax):
           Nodes_Filter[i].neighbors.append(Nodes_Filter[o]) 
           adj_matrix_SUPLE.append(1)
         else:
            adj_matrix_SUPLE.append(0)
       else:
        adj_matrix_SUPLE.append(0)
     adj_matrix2.append(adj_matrix_SUPLE)          
  
  for i in range(len(Nodes_Filter)):
    all_nodes.append(Nodes_Filter[i])
    
  total_cliques = []
  NumNodes = {
    i: set(num for num, j in enumerate(row) if j)
    for i, row in enumerate(adj_matrix2)
  }
  P2 = NumNodes.keys()

  optimized_cliques_list = list(Optimizer_V2(bron_kerbosch(adj_matrix2, pivot=True),3))
  new_width = width*0.5*2
  new_height = height*0.5*2 
  number_files_ours_aux = 0

  number_files_ours_aux = 0
  shutil.rmtree(image_save+"/AUX/")
  os.makedirs(image_save+"/AUX/")
  for ix in optimized_cliques_list:

    coordinate_selected_list = []
    for ab in ix:
     coordenadaxyc =all_nodes[ab].coordenadas2
     coordinate_selected_list.append(coordenadaxyc)
    
    if len(coordinate_selected_list) > 0:
        number_files_ours_aux = number_files_ours_aux + 1
        centroide = centroid1(coordinate_selected_list)
        a1 = centroide[0]
        a2 = centroide[1]
        RECORTEX = int(new_width/2)
        RECORTEY = int(new_height/2)
        
        im_crop_outside = im.crop((a1-RECORTEX, a2-RECORTEY, a1+RECORTEX, a2+RECORTEY))
        nombre_sr_2 =image_save+"AUX/{}.png".format(str(new_counter))
        #print(type(im_crop_outside))
        #im_crop_outside.save(nombre_sr_2, quality=100)
      
        RECORTEX = int(width/2)
        RECORTEY = int(height/2)  

        'INFERENCE'
        
        make_simple_inference240(im_crop_outside,new_counter,a1-RECORTEX,a2-RECORTEY,
                                           diccionario_final_predicciones,
                                           diccionario_final_scores,
                                           diccionario_final_clases,
                                           diciconario_final_mascaras,
                                           width,height,counter,LISTA_TRIPLAS,image_save+'/FILES/',scorethresh,classeslist,image_path)
        
        
      
      
        
        new_counter = new_counter+1
    
    
    
    
  PATH_SR_V2  = '/opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png'
  PATH_RAW_V2 = pather_raw
  
  
  fixed_points = generate_extra_points_V2(PATH_RAW_V2, PATH_SR_V2)
  string_extra = 'EXTRA-'


  for id, point in enumerate(fixed_points):
        im_crop_outside = im.crop(point)
        nombre_sr_332 = image_save+"/AUX/{}.jpg".format(str(string_extra+str(id)))
        #im_crop_outside.save(nombre_sr_332, quality=100)
        a1e = point[0]
        a2e = point[1]
        
        
        new_counter_name = string_extra+str(new_counter)
        
        make_simple_inference240(im_crop_outside,new_counter_name,a1e,a2e,
                                            diccionario_final_predicciones,
                                           diccionario_final_scores,
                                           diccionario_final_clases,
                                           diciconario_final_mascaras,
                                           width,height,counter,LISTA_TRIPLAS,image_save+'/FILES/',scorethresh,classeslist,image_path)

        
        
        new_counter = new_counter+1
        
  number_files_ours = number_files_ours_aux + 5
    
    

  detections['detection_boxes'] = np.array(diccionario_final_predicciones)
  detections['detection_classes'] = diccionario_final_clases
  detections['detection_scores'] = diccionario_final_scores
  detections['detection_masks'] = diciconario_final_mascaras

  

  
  diccionario_final2_predicciones = []
  diccionario_final2_scores = []
  diccionario_final2_clases = []
  diccionario_final2_mascaras = []

  puntero = 0
  error = [-1,-1,-1,-1]
    
  print('TRIPLASSSS ',len(LISTA_TRIPLAS))
  cluster_final = create_clusters(LISTA_TRIPLAS, threshold =0)
  print('CLUSTERSSS ',len(cluster_final))

  '''LLEGADOS A ESTE PUNTO TENGO QUE HACER IOU'''
    
    
  LISTA_BOXES_OK = []
  LISTA_CLASES_OK = []
  LISTA_SCORES_OK = []
  LISTA_MASCARAS_OK = []
    
  for indexcluster in range(len(cluster_final)):
    tripla = cluster_final[indexcluster]
    
    
    
    lista_mascaras_lista = tripla[0]
    lista_clases_lista = tripla[1].tolist()
    lista_scores_lista = tripla[2].tolist()
   
    
    '''print(lista_clases_lista)'''
    
    if len(lista_mascaras_lista) > 0:
        #print('UNIFICANDO CON ',mascarathr)
        indice = np.argmax(lista_scores_lista)
        mascara_unificada = unified_mask(lista_mascaras_lista, width, height, mascarathr)
        #mascara_unificada = lista_mascaras_lista[indice]
       
        #print(type(lista_mascaras_lista[indice]))

        
        ground_truth_binary_mask = mascara_unificada
        fortran_ground_truth_binary_mask = np.asfortranarray(ground_truth_binary_mask)
        encoded_ground_truth = mask.encode(fortran_ground_truth_binary_mask)
        ground_truth_bounding_box = mask.toBbox(encoded_ground_truth)
        listauno = ground_truth_bounding_box.tolist()
        #coordenadas_ejemplo = calculatemask(mascara_unificada)
        
        ground_truth_area = mask.area(encoded_ground_truth)
        

        if int(ground_truth_area.tolist())> 0.1:
        
            #listauno = mask_to_bbox22(mascara_unificada)
            LISTA_BOXES_OK.append([listauno[1]/height,listauno[0]/width,(listauno[1]+listauno[3])/height,(listauno[0]+listauno[2])/width])

            #LISTA_BOXES_OK.append(mask_to_bbox22(mascara_unificada))
            MOST_FRECUENT_CLASS = most_frequent(lista_clases_lista)
            LISTA_CLASES_OK.append(MOST_FRECUENT_CLASS)
            LISTA_SCORES_OK.append(lista_scores_lista[indice])
            LISTA_MASCARAS_OK.append(mascara_unificada)
    
    
  
  print('TENGO QUE IMPRIMIR ',len(LISTA_BOXES_OK))
  #print('JI')
  #print(LISTA_BOXES_OK)
  #print('JI')
  #print(LISTA_CLASES_OK)
  #print('JI')
  #print(LISTA_SCORES_OK)
  #print('JI')
  
            
  detections['detection_boxes'] = np.array(LISTA_BOXES_OK)
  detections['detection_classes'] = np.array(LISTA_CLASES_OK)
  detections['detection_scores'] = np.array(LISTA_SCORES_OK)
  detections['detection_masks_reframed'] = np.array(LISTA_MASCARAS_OK)
  

  boxes_filter = []
  scores_filter = []
  classes_filter = []
  mask_filter = []

  for item in range(len(detections['detection_boxes'])):
    if detections['detection_scores'][item] > scorethresh:
      boxes_filter.append(detections['detection_boxes'][item].tolist())
      scores_filter.append(detections['detection_scores'][item])
      classes_filter.append(detections['detection_classes'][item])
      mask_filter.append(detections['detection_masks_reframed'][item])

  print('TENGO QUE IMPRIMIR ',len(boxes_filter))
  #boxes_filter, classes_filter, scores_filter, mask_filter = filterdual(boxes_filter, classes_filter, scores_filter, mask_filter)

  detections['detection_boxes'] = np.array(boxes_filter)
  detections['detection_classes'] = np.array(classes_filter)
  detections['detection_scores'] = np.array(scores_filter)
  detections['detection_masks_reframed'] = np.array(mask_filter)
    
 

  
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image2,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      instance_masks=detections.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      max_boxes_to_draw=300,
      min_score_thresh=scorethresh,
      agnostic_mode=False,
      line_thickness=1)
  

  '''
  display(Image.fromarray(image2))
  '''

  plt.axis('off')
  plt.imshow(image2)
  
  
  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']
  mascaras_finales = detections['detection_masks_reframed']

  min_score_thresh=scorethresh
  img = cv2.imread(image_path)
  nombre = counter
  dpi = 120
  #dpi = 300
  plt.savefig(image_save+'/RESULTADOS_SR/'+nombre,  dpi=dpi ,bbox_inches='tight',pad_inches = 0)
  plt.clf()
  

  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']

  min_score_thresh = scorethresh
  salida = []
  salida.append(boxes.tolist())
  salida.append(scores[0])
  salida.append(clases_detected) 

  image = Image.open(image_path)
  width, height = image.size

  salida.append(width)
  salida.append(height)
  salida.append(mascaras_finales)

  con_propuesta = len(boxes.tolist())
  

  salida.append(sin_propuesta_2)
  salida.append(number_files_ours)

  
  return salida

def most_frequent(List):
    counter = 0
    num = List[0]
     
    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i
 
    return num


def unified_mask(list_mask, width, height, threshold):
    numero_mascaras = len(list_mask)
    threshold_good = numero_mascaras * threshold / 100 
    print(numero_mascaras,threshold,' UNIFICANDO MASCARA CON THRESHOLD ',threshold_good)
    if threshold_good > 2:
        threshold_good = 2
    mascara_unificada = np.zeros((height,width),dtype=np.uint8)
   
    for selected_mask in list_mask:
        mascara_unificada = mascara_unificada + selected_mask
    
    result = np.where(mascara_unificada>threshold_good, 1, 0)
    result_final = result.astype(np.uint8)
    print('FIN DE LA UNIFICACION')
    return result_final

def masktoBbox(mask):
    [x,y,w,h] = COCOmask.toBbox(mask)
    return np.asarray([x,y, x+w, y+h]) 

def calculatemask(masks):
    pos = np.where(masks)             
    xmin = np.min(pos[1])             
    xmax = np.max(pos[1])             
    ymin = np.min(pos[0])             
    ymax = np.max(pos[0])             
    
    return [ymin, xmin, ymax+ymin, xmax+xmin]

def mask_to_bbox22(mask):
    mask = np.where(mask>0, 255, 0)
    bboxes = []

    mask = mask_to_border22(mask)
    lbl = label(mask)
    prop = regionprops(lbl)[0]
    
    x1a = prop.bbox[1]
    y1a = prop.bbox[0]
    x2a = prop.bbox[3]
    y2a = prop.bbox[2]


    return [x1a, y1a, x2a, y2a]

def mask_to_border22(mask):
    h, w = mask.shape
    border = np.zeros((h, w))

    contours = find_contours(mask, 128)
    for contour in contours:
        for c in contour:
            x = int(c[0])
            y = int(c[1])
            border[x][y] = 255

    return border

def extract_bboxes33(mask):
    """Compute bounding boxes from masks.
    mask: [height, width, num_instances]. Mask pixels are either 1 or 0.
    Returns: bbox array [num_instances, (y1, x1, y2, x2)].
    """
    boxes = np.zeros([mask.shape[-1], 4], dtype=np.int32)
    for i in range(mask.shape[-1]):
        m = mask[:, :, i]
        # Bounding box.
        horizontal_indicies = np.where(np.any(m, axis=0))[0]
        vertical_indicies = np.where(np.any(m, axis=1))[0]
        if horizontal_indicies.shape[0]:
            x1, x2 = horizontal_indicies[[0, -1]]
            y1, y2 = vertical_indicies[[0, -1]]
            # x2 and y2 should not be part of the box. Increment by 1.
            x2 += 1
            y2 += 1
        else:
            # No mask for this instance. Might happen due to
            # resizing or cropping. Set bbox to zeros
            x1, x2, y1, y2 = 0, 0, 0, 0
        boxes[i] = np.array([y1, x1, y2, x2])
    return boxes.astype(np.int32)

In [ ]:
tesla

In [16]:

MODELO_ELEGIDO = 'SR-DIRECT-MASK-MUNSTER'
array_veces = []
window = 'test'
create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentos/'+MODELO_ELEGIDO+'/'+str(window)+'/')
create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentos/'+MODELO_ELEGIDO+'/'+str(window)+"/SALIDA/")
create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentos/'+MODELO_ELEGIDO+'/'+str(window)+"/AUX/")
create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentos/'+MODELO_ELEGIDO+'/'+str(window)+"/AUX_COMPLEJO/")
create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentos/'+MODELO_ELEGIDO+'/'+str(window)+"/JSON/")
create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentos/'+MODELO_ELEGIDO+'/'+str(window)+"/TEMP/")
create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentos/'+MODELO_ELEGIDO+'/'+str(window)+'/RESULTADOS_SR/')
create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentos//'+MODELO_ELEGIDO+'/'+str(window)+'/AUX/')
create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentos//'+MODELO_ELEGIDO+'/'+str(window)+'/FILES/')
TAM_VENTANA = 0



#PENDIENTES

#SOLUCIONADOS
#munster/munster_000121_000019_leftImg8bit.png
#munster/munster_000001_000019_leftImg8bit.png
#munster/munster_000081_000019_leftImg8bit.png tarda mucho ;( y resultados regu

#munster_000000_000019_leftImg8bit.png
#munster_000168_000019_leftImg8bit.png
#munster_000125_000019_leftImg8bit.png
#munster_000151_000019_leftImg8bit.png


make_inference240('/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/zurich/zurich_000002_000019_leftImg8bit.png',
                  'ours95.png','/opt/share/IWINAC-EXTENDED-FOLDER/experimentos/'+MODELO_ELEGIDO+'/'+str(window)+'/',
                  0,
                  20,
                  TAM_VENTANA,
                  0.7,
                  [1,2,3,4,8]
)


#make_inference240RAW('/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/munster/munster_000168_000019_leftImg8bit.png','munster_000168_000019_leftImg8bit.png','/opt/share/IWINAC-EXTENDED-FOLDER/experimentos/'+MODELO_ELEGIDO+'/'+str(window)+'/',0,25,0.7,[1,2,3,4,8])


Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/zurich/zurich_000002_000019_leftImg8bit.png... 
TRIPLASSSS  20
HE METIDO UNOOOOOO
STARTTT  5
0.0
0.0
0.2744913807980672
0.7900096325856611
0.0
0  NO ESTA EN  [3]
1  NO ESTA EN  [3]
2  NO ESTA EN  [3]
4  NO ESTA EN  [3]
ENDDDD  1
ESTAS SON LAS COINCIDENCIAS  1
AVERRR  4
HAY  20  LISTAS
STARTTT  8
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0  NO ESTA EN  []
1  NO ESTA EN  []
2  NO ESTA EN  []
3  NO ESTA EN  []
4  NO ESTA EN  []
5  NO ESTA EN  []
6  NO ESTA EN  []
7  NO ESTA EN  []
ENDDDD  0
ESTAS SON LAS COINCIDENCIAS  0
AVERRR  8
HAY  20  LISTAS
STARTTT  2
0.7736282733827083
0.28388998035363455
1  NO ESTA EN  [0]
ENDDDD  1
ESTAS SON LAS COINCIDENCIAS  1
AVERRR  1
HAY  20  LISTAS
STARTTT  9
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0  NO ESTA EN  []
1  NO ESTA EN  []
2  NO ESTA EN  []
3  NO ESTA EN  []
4  NO ESTA EN  []
5  NO ESTA EN  []
6  NO ESTA EN  []
7  NO ESTA EN  []
8  NO ESTA EN  []
ENDDDD  0
ESTAS SON LA

HE METIDO UNOOOOOO
STARTTT  2
0.0
0.9223066938624143
0  NO ESTA EN  [1]
ENDDDD  1
ESTAS SON LAS COINCIDENCIAS  1
AVERRR  1
HAY  16  LISTAS
STARTTT  7
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0  NO ESTA EN  []
1  NO ESTA EN  []
2  NO ESTA EN  []
3  NO ESTA EN  []
4  NO ESTA EN  []
5  NO ESTA EN  []
6  NO ESTA EN  []
ENDDDD  0
ESTAS SON LAS COINCIDENCIAS  0
AVERRR  7
HAY  16  LISTAS
STARTTT  8
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0  NO ESTA EN  []
1  NO ESTA EN  []
2  NO ESTA EN  []
3  NO ESTA EN  []
4  NO ESTA EN  []
5  NO ESTA EN  []
6  NO ESTA EN  []
7  NO ESTA EN  []
ENDDDD  0
ESTAS SON LAS COINCIDENCIAS  0
AVERRR  8
HAY  16  LISTAS
STARTTT  7
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0  NO ESTA EN  []
1  NO ESTA EN  []
2  NO ESTA EN  []
3  NO ESTA EN  []
4  NO ESTA EN  []
5  NO ESTA EN  []
6  NO ESTA EN  []
ENDDDD  0
ESTAS SON LAS COINCIDENCIAS  0
AVERRR  7
HAY  16  LISTAS
STARTTT  8
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0  NO ESTA EN  []
1  NO ESTA EN  []
2  NO ESTA EN  []
3  NO ESTA EN  []
4  NO ESTA EN  []
5  NO ESTA

0.0
0.0
0.9837837837837837
0  NO ESTA EN  [5]
1  NO ESTA EN  [5]
2  NO ESTA EN  [5]
3  NO ESTA EN  [5]
4  NO ESTA EN  [5]
ENDDDD  1
ESTAS SON LAS COINCIDENCIAS  1
AVERRR  5
HAY  10  LISTAS
STARTTT  5
0.0
0.0
0.0
0.0
0.0
0  NO ESTA EN  []
1  NO ESTA EN  []
2  NO ESTA EN  []
3  NO ESTA EN  []
4  NO ESTA EN  []
ENDDDD  0
ESTAS SON LAS COINCIDENCIAS  0
AVERRR  5
HAY  10  LISTAS
METO  8
HE METIDO UNOOOOOO
STARTTT  6
0.9577142857142857
0.0
0.0
0.0
0.0
0.0
1  NO ESTA EN  [0]
2  NO ESTA EN  [0]
3  NO ESTA EN  [0]
4  NO ESTA EN  [0]
5  NO ESTA EN  [0]
ENDDDD  1
ESTAS SON LAS COINCIDENCIAS  1
AVERRR  5
HAY  10  LISTAS
STARTTT  6
0.9943995021779714
0.0
0.0
0.0
0.0
0.0
1  NO ESTA EN  [0]
2  NO ESTA EN  [0]
3  NO ESTA EN  [0]
4  NO ESTA EN  [0]
5  NO ESTA EN  [0]
ENDDDD  1
ESTAS SON LAS COINCIDENCIAS  1
AVERRR  5
HAY  10  LISTAS
STARTTT  5
0.9724501758499414
0.0
0.0
0.0
0.0
1  NO ESTA EN  [0]
2  NO ESTA EN  [0]
3  NO ESTA EN  [0]
4  NO ESTA EN  [0]
ENDDDD  1
ESTAS SON LAS COINCIDENCIAS  1
AVERRR  4

0.8816326530612245
0.0
1  NO ESTA EN  [0]
ENDDDD  1
ESTAS SON LAS COINCIDENCIAS  1
AVERRR  1
HAY  8  LISTAS
METO  9
HE METIDO UNOOOOOO
STARTTT  1
0.7348837209302326
ENDDDD  1
ESTAS SON LAS COINCIDENCIAS  1
AVERRR  0
HAY  1  LISTAS
METO  2
CLUSTERSSS  15
9 20  UNIFICANDO MASCARA CON THRESHOLD  1.8
FIN DE LA UNIFICACION
9 20  UNIFICANDO MASCARA CON THRESHOLD  1.8
FIN DE LA UNIFICACION
12 20  UNIFICANDO MASCARA CON THRESHOLD  2.4
FIN DE LA UNIFICACION
5 20  UNIFICANDO MASCARA CON THRESHOLD  1.0
FIN DE LA UNIFICACION
5 20  UNIFICANDO MASCARA CON THRESHOLD  1.0
FIN DE LA UNIFICACION
1 20  UNIFICANDO MASCARA CON THRESHOLD  0.2
FIN DE LA UNIFICACION
2 20  UNIFICANDO MASCARA CON THRESHOLD  0.4
FIN DE LA UNIFICACION
8 20  UNIFICANDO MASCARA CON THRESHOLD  1.6
FIN DE LA UNIFICACION
9 20  UNIFICANDO MASCARA CON THRESHOLD  1.8
FIN DE LA UNIFICACION
7 20  UNIFICANDO MASCARA CON THRESHOLD  1.4
FIN DE LA UNIFICACION
6 20  UNIFICANDO MASCARA CON THRESHOLD  1.2
FIN DE LA UNIFICACION
8 20  UNIFICANDO MA

KeyboardInterrupt: 

In [122]:
datasets = os.listdir('/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster')
for data in datasets:
    print(data)

munster_000118_000019_leftImg8bit.png
munster_000145_000019_leftImg8bit.png
munster_000076_000019_leftImg8bit.png
munster_000168_000019_leftImg8bit.png
munster_000113_000019_leftImg8bit.png
munster_000010_000019_leftImg8bit.png
munster_000073_000019_leftImg8bit.png
munster_000125_000019_leftImg8bit.png
munster_000039_000019_leftImg8bit.png
munster_000135_000019_leftImg8bit.png
munster_000070_000019_leftImg8bit.png
munster_000121_000019_leftImg8bit.png
munster_000045_000019_leftImg8bit.png
munster_000133_000019_leftImg8bit.png
munster_000136_000019_leftImg8bit.png
munster_000129_000019_leftImg8bit.png
munster_000148_000019_leftImg8bit.png
munster_000157_000019_leftImg8bit.png
munster_000040_000019_leftImg8bit.png
munster_000107_000019_leftImg8bit.png
munster_000057_000019_leftImg8bit.png
munster_000100_000019_leftImg8bit.png
munster_000124_000019_leftImg8bit.png
munster_000127_000019_leftImg8bit.png
munster_000095_000019_leftImg8bit.png
munster_000159_000019_leftImg8bit.png
munster_0000

In [41]:
shutil.rmtree('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/')

In [42]:
os.mkdir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/')

In [14]:
%%capture
from pycocotools import mask
from skimage import measure

diccionario_data_gt = {
    'zurich':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/zurich_122_instancesonly_filtered_gtFine_val.json',
    'weimar':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/weimar_142_instancesonly_filtered_gtFine_val.json',
    'ulm':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/ulm_95_instancesonly_filtered_gtFine_val.json',
    'tubingen':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/tubingen_144_instancesonly_filtered_gtFine_val.json',
    'stuttgart':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/stuttgart_196_instancesonly_filtered_gtFine_val.json',
    'strasbourg':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/strasbourg_365_instancesonly_filtered_gtFine_val.json',
    'munster':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/munster_174_instancesonly_filtered_gtFine_val.json',
    'monchengladbach':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/monchengladbach_94_instancesonly_filtered_gtFine_val.json',
    'lindau':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/lindau_59_instancesonly_filtered_gtFine_val.json',
    'krefeld':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/krefeld_99_instancesonly_filtered_gtFine_val.json',
    'jena':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/jena_119_instancesonly_filtered_gtFine_val.json',
    'hanover':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/hanover_196_instancesonly_filtered_gtFine_val.json',
    'hamburg':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/hamburg_248_instancesonly_filtered_gtFine_val.json',
    'frankfurt':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/frankfurt_267_instancesonly_filtered_gtFine_val.json',
    'erfurt':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/erfurt_109_instancesonly_filtered_gtFine_val.json',
    'dusseldorf':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/dusseldorf_221_instancesonly_filtered_gtFine_val.json',
    'darmstadt':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/darmstadt_85_instancesonly_filtered_gtFine_val.json',
    'cologne':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/cologne_154_instancesonly_filtered_gtFine_val.json',
    'bremen':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/bremen_316_instancesonly_filtered_gtFine_val.json',
    'bochum':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/bochum_96_instancesonly_filtered_gtFine_val.json',
    'aachen':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/aachen_174_instancesonly_filtered_gtFine_val.json',
    'test':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/test.json'
}


datasets = os.listdir('/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/')

datasets = ['zurich','ulm','munster','monchengladbach','lindau','krefeld','jena','erfurt','darmstadt','bochum','frankfurt']
#datasets2 = ['weimar', 'tubingen', 'stuttgart', 'hanover', 'cologne', 'aachen' ,'dusseldorf' ,'hamburg' ,'bremen', 'strasbourg']
#datasets = ['lindau']
datasets = ['zurich']

for selected_dataset in datasets:
    
    if selected_dataset in diccionario_data_gt:


        jsonString_mod = diccionario_data_gt[selected_dataset]
        diccionario_ids_mod = {}

        
        with open(jsonString_mod) as json_file_mod:
            data_mod = json.load(json_file_mod)
            for p3_mod in data_mod['images']:                  
                data_s_image_mod = p3_mod['file_name'].replace('leftImg8bit/val/'+str(selected_dataset)+'/','')
                #data_s_image_mod = p3_mod['file_name'].replace('leftImg8bit/val/munster/','')
                data_s_id_mod = p3_mod['id']
                diccionario_ids_mod[data_s_image_mod] = data_s_id_mod

        #MASKRCNN-EXTRA-HOTFIX-
        MODELO_ELEGIDO = 'MASKRCNN-HOTFIX-OURPROPOSAL-'+str(selected_dataset)

        imagenes_dir2 = '/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/'+str(selected_dataset)
        imagenes_dir3 = '/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/'+str(selected_dataset)

        contenido2 = os.listdir(imagenes_dir2)
        imagenes_dir2 = []
        counter = 0
        result = []
        result2 = []


        #PLEASE INSERT THE MINIMUN SCORE TO DETECT ONE OBJECT


        #PLEASE INSERT % OF THE WINDOWS SIZE R TO BE EVALLUATED
        WSizeR =  [0, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 1]
        #WSizeR = [1, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10, 0]
        #WSizeR = [0]


        for window in WSizeR:


            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev2/'+MODELO_ELEGIDO+'/'+str(window)+'/')
            #create_dir('/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS_SR/')
            #create_dir('usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/AUX/')
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev2/'+MODELO_ELEGIDO+'/'+str(window)+"/SALIDA/")
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev2/'+MODELO_ELEGIDO+'/'+str(window)+"/AUX/")
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev2/'+MODELO_ELEGIDO+'/'+str(window)+"/AUX_COMPLEJO/")
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev2/'+MODELO_ELEGIDO+'/'+str(window)+"/JSON/")
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev2/'+MODELO_ELEGIDO+'/'+str(window)+"/TEMP/")
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev2/'+MODELO_ELEGIDO+'/'+str(window)+'/RESULTADOS_SR/')
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev2//'+MODELO_ELEGIDO+'/'+str(window)+'/AUX/')
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev2//'+MODELO_ELEGIDO+'/'+str(window)+'/FILES/')




            tiempos_procesamiento2 = []
            array_veces = []
            tiempos_procesar2 =  []

            out = []
            out2 = []
            concon = 0
            result2 = []
            out222 = []

            out5 = []


            '''RECORRO LAS IMÁGENES ALMACENADAS EN UN DIRECTORIO'''
            for fichero2 in contenido2:
              #x = re.search("[0-9]+[0-9]*", fichero2)
              #id = x.sgroup(0)
              image_path_gg = imagenes_dir3+"/"+str(fichero2)
              print(image_path_gg)
              contador_veces = 0

              salida = make_inference240(image_path_gg,fichero2,'/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev2/'+MODELO_ELEGIDO+'/'+str(window)+'/',0,20,window,0.7,[1,2,3,4,8])
              #salida = make_inference240RAW(image_path_gg,fichero2,'/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev2/'+MODELO_ELEGIDO+'/'+str(window)+'/',0,20,0.7,[1,2,3,4,8])


              print('he terminado')
              counter = counter+1

              result2 = []
              result3 = []
              out3 = []

              '''OBTENGO EL ÍNDICE PERTENECIENTE A DICHA IMAGEN'''
              #converted_num = int(id)
              converted_num = diccionario_ids_mod[fichero2]

              width = salida[3]
              height = salida[4]
              mascaras_finales = salida[5]
              boxes2 = []

              '''REALIZO LA TRANSOFORMACIÓN DE COORDENADAS DE ACUERDO CON EL FORMATO DE COCO'''
              contador_mascara = 0
              punterog = 0
              result = []
              result2 = []
              out39 = []
              result39 = []


              cluster_bueno = []
              numero_subimagenes = salida[7]
              numero_subimagenes_sin_propuesta = salida[6]

              '''
              for indexcluster in range(len(salida[5])):
                tripla = salida[5][indexcluster]

                lista_boxes_lista = tripla[0].tolist()
                lista_clases_lista = tripla[1].tolist()
                lista_scores_lista = tripla[2].tolist()
                cluster_aux = []
                cluster_aux.append(lista_boxes_lista)
                cluster_aux.append(lista_clases_lista)
                cluster_aux.append(lista_scores_lista)
                cluster_bueno.append(cluster_aux)

                "triplas": cluster_bueno,
              '''
              result2.extend(
                    [
                        {
                            "image_id": converted_num,
                            "numero_subimagenes": int(numero_subimagenes),
                            "numero_subimagenes_sin_propuesta":numero_subimagenes_sin_propuesta,
                        }
                    ]
              )


              for box in salida[0]:


                ymin = int(box[0]*height)
                xmin = int(box[1]*width)
                ymax = int(box[2]*height)
                xmax = int(box[3]*width) 

                box_new = []
                box_new.append(xmin)
                box_new.append(ymin)
                box_new.append(xmax-xmin)
                box_new.append(ymax-ymin)




                ground_truth_binary_mask = mascaras_finales[contador_mascara]
                contador_mascara = contador_mascara + 1

                fortran_ground_truth_binary_mask = np.asfortranarray(ground_truth_binary_mask)
                encoded_ground_truth = mask.encode(fortran_ground_truth_binary_mask)
                ground_truth_area = mask.area(encoded_ground_truth)
                ground_truth_bounding_box = mask.toBbox(encoded_ground_truth)
                contours = measure.find_contours(ground_truth_binary_mask, 0.5)



                annotation = {
                    "image_id": 1,
                    "category_id": 1,
                    "bbox": [],
                    "score": 1,
                    "segmentation": [],
                    "area": []
                }

                annotation["image_id"]= converted_num
                annotation["category_id"]= int(salida[2][punterog])
                #cambiar box_new por box para RAW
                annotation["bbox"]= ground_truth_bounding_box.tolist()
                annotation["score"]= salida[1][punterog].astype(float)
                annotation["area"]= ground_truth_area.tolist()
                punterog = punterog + 1

                for contour in contours:
                    contour = np.flip(contour, axis=1)
                    segmentation = contour.ravel().tolist()
                    annotation["segmentation"].append(segmentation)

                if len(annotation["segmentation"]) > 0:
                        result.extend([annotation])
                        result39.extend([annotation])


                '''print(annotation)'''
              '''
              for sample39 in result39:    
                out39.append(sample39)

              nombre_json = '/opt/share/IWINAC-EXTENDED-FOLDER/experimentos/'+MODELO_ELEGIDO+'/'+str(window)+'/JSON/'+str(converted_num)+'.json'
              with open(nombre_json, 'w', encoding='utf-8') as file39:
                json.dump(out39, file39, ensure_ascii=False)
              '''

              for sample in result:    
                out5.append(sample)

              for sample2 in result2:    
                out222.append(sample2)

              '''GUARDO EL JSON EN UN ARCHIVO'''
            with open('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev2/'+MODELO_ELEGIDO+'/'+str(window)+'/'+'test_data_modificado.json', 'w', encoding='utf-8') as file:
                json.dump(out5, file, ensure_ascii=False)

            with open('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev2/'+MODELO_ELEGIDO+'/'+str(window)+'/'+'test_data_modificado2.json', 'w', encoding='utf-8') as file2:
                json.dump(out222, file2, ensure_ascii=False)



KeyboardInterrupt: 

In [13]:
!pip install scikit-image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 8.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.3 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 8.4 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.3/210.3 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 8.6 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 9.1 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [15]:
from scipy.ndimage import zoom

In [57]:
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

annType = ['segm','bbox','keypoints']
annType = annType[0]      #specify type here
prefix = 'person_keypoints' if annType=='keypoints' else 'instances'
print ('Running demo for *%s* results.'%(annType))

'''CARGO EL JSON CON LOS GROUND TRUTH (GT)'''
#annFile = '/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/munster_174_instancesonly_filtered_gtFine_test.json'
annFile = '/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/ulm_95_instancesonly_filtered_gtFine_val.json'
cocoGt=COCO(annFile)

'''CARGO EL JSON CON LAS DETECCIONES OBTENIDAS CON LA PROPUESTA EXCLUSIVA DE SR'''
resFile = '/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/MASKRCNN-HOTFIX-DIRECTSR-NEWMASKOK-ulm/0/test_data_modificado.json'
#resFile = '/opt/share/IWINAC-EXTENDED-FOLDER/experimentos/MASKRCNN-EXTRA-HOTFIX-RAW--munster/0/test_data_modificado.json'
cocoDt=cocoGt.loadRes(resFile)

dts = json.load(open(resFile,'r'))
imgIds = [imid['image_id'] for imid in dts]
imgIds = sorted(list(set(imgIds)))

cocoEval = COCOeval(cocoGt,cocoDt,annType)
cocoEval.params.imgIds  = imgIds
'''SELECCIONO LA CLASE 3 CORRESPONDIENTE CON LOS COCHES'''
cocoEval.params.catIds = [1,2,3,4,8] 

cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running demo for *segm* results.
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.09s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.25s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.196
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.345
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.015
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.179
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.459
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.131
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.238
 Average Recall     (AR) @[ IoU=

In [ ]:
##ZURICH
#RAW 0.133 0.247 0.125 0.006 0.118 0.375

#WINDOWS
#0.0 0.214 0.419 0.210 0.056 0.216 0.446
#0.1 0.210 0.424 0.200 0.059 0.216 0.440
#0.2 0.212 0.419 0.210 0.045 0.222 0.444
#0.3 
#0.4 
#0.5 
#0.6 0.212 0.422 0.206 0.054 0.212 0.453
#0.7 
#0.8 
#0.9 
#1.0 0.202 0.388 0.199 0.042 0.201 0.440 


##ULM
#RAW 0.194 0.341 0.191 0.017 0.181 0.455

#WINDOWS
#0.0 0.261 0.462 0.256 0.071 0.280 0.491
#0.1 0.261 0.470 0.248 0.058 0.289 0.492
#0.2 0.264 0.484 0.247 0.082 0.284 0.493
#0.3 0.268 0.479 0.253 0.068 0.293 0.496
#0.4 0.254 0.452 0.240 0.058 0.286 0.476
#0.5 0.256 0.465 0.240 0.057 0.281 0.492
#0.6 0.261 0.478 0.246 0.068 0.285 0.477
#0.7 0.258 0.465 0.246 0.068 0.282 0.475
#0.8 0.257 0.464 0.244 0.058 0.290 0.477
#0.9 0.255 0.463 0.236 0.056 0.273 0.479
#1.0 0.256 0.449 0.251 0.065 0.271 0.482


##LINDAU
#RAW 0.299 0.537 0.192 0.024 0.247 0.601

#WINDOWS
#0.0 0.404 0.620 0.524 0.068 0.394 0.665
#0.1 0.403 0.612 0.518 0.070 0.394 0.674
#0.2 0.385 0.608 0.458 0.068 0.377 0.636 
#0.3 0.393 0.626 0.488 0.068 0.389 0.646
#0.4 0.393 0.624 0.466 0.068 0.384 0.647
#0.5 0.404 0.617 0.465 0.065 0.396 0.629
#0.6 0.406 0.623 0.465 0.068 0.399 0.610
#0.7 0.386 0.620 0.467 0.061 0.376 0.637
#0.8 0.379 0.609 0.462 0.060 0.372 0.611
#0.9 0.378 0.611 0.460 0.065 0.368 0.613
#1.0 0.373 0.605 0.451 0.064 0.362 0.602








In [11]:
def make_inference240RAW(image_path,counter,image_save,contador_veces,mascarathr,scorethresh,classeslist):
  print('Running inference for {}... '.format(image_path), end='\n')
  
  image_np = load_image_into_numpy_array(image_path)
  image = image_path
    
    
  np.set_printoptions(threshold=np.inf)
  dpi=300
  model = detect_fn
  image_np = load_image_into_numpy_array(image)
  image2 = np.array(Image.open(image))
  image = np.array(Image.open(image))
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis, ...]

  model_fn = model.signatures['serving_default']
  detections = model_fn(input_tensor)
  num_detections = int(detections.pop('num_detections'))
  need_detection_key = ['detection_classes','detection_boxes','detection_masks','detection_scores']
  detections = {key: detections[key][0, :num_detections].numpy()
                for key in need_detection_key}
  detections['num_detections'] = num_detections
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
  if 'detection_masks' in detections:
      detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
          tf.convert_to_tensor(detections['detection_masks']), detections['detection_boxes'],
          image.shape[0], image.shape[1])
      detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                        tf.uint8)
      detections['detection_masks_reframed'] = detection_masks_reframed.numpy()
  image_np_with_detections = image_np.copy()



  boxes_filter = []
  scores_filter = []
  classes_filter = []
  mask_filter = []

  for item in range(len(detections['detection_boxes'])):
        if detections['detection_classes'][item] in classeslist and detections['detection_scores'][item]>scorethresh:
            #boxes_filter.append(detections['detection_boxes'][item].tolist())
            
            cajas = detections['detection_boxes'][item].tolist()
            boxeada = []
            for element in cajas:
                boxeada.append(element/2)
              

            boxes_filter.append(boxeada)
            
            downscaled_mask = zoom(detections['detection_masks_reframed'][item], zoom=0.5, order=0)
        
            scores_filter.append(detections['detection_scores'][item])
            classes_filter.append(detections['detection_classes'][item])
            #mask_filter.append(detections['detection_masks_reframed'][item])
            mask_filter.append(downscaled_mask)
            
  #boxes_filter, classes_filter, scores_filter, mask_filter = filterdual(boxes_filter, classes_filter, scores_filter, mask_filter)
   
  detections['detection_boxes'] = np.array(boxes_filter)
  detections['detection_classes'] = np.array(classes_filter)
  detections['detection_scores'] = np.array(scores_filter)
  detections['detection_masks_reframed'] = np.array(mask_filter)



  min_score_thresh=scorethresh
  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']
  mask_detected = detections['detection_masks_reframed']

  true_boxes  = boxes[scores[0] > min_score_thresh]
  true_scores = scores[0][scores[0] > min_score_thresh]
  true_clases = clases_detected[scores[0] > min_score_thresh]
  true_mask = mask_detected[scores[0] > min_score_thresh]
  
          
  detections['detection_boxes'] = np.array(true_boxes)
  detections['detection_classes'] = np.array(true_clases)
  detections['detection_scores'] = np.array(true_scores)
  detections['detection_masks_reframed'] = np.array(true_mask)
  

  boxes_filter = []
  scores_filter = []
  classes_filter = []
  mask_filter = []

  for item in range(len(detections['detection_boxes'])):
    if detections['detection_scores'][item] > scorethresh:
      boxes_filter.append(detections['detection_boxes'][item].tolist())
      scores_filter.append(detections['detection_scores'][item])
      classes_filter.append(detections['detection_classes'][item])
      mask_filter.append(detections['detection_masks_reframed'][item])


  detections['detection_boxes'] = np.array(boxes_filter)
  detections['detection_classes'] = np.array(classes_filter)
  detections['detection_scores'] = np.array(scores_filter)
  detections['detection_masks_reframed'] = np.array(mask_filter)
    
 

  '''
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image2,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      instance_masks=detections.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      max_boxes_to_draw=300,
      min_score_thresh=scorethresh,
      agnostic_mode=False,
      line_thickness=1)
  '''

  '''
  display(Image.fromarray(image2))
  '''
  plt.axis('off')
  plt.imshow(image2)
  
  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']
  mascaras_finales = detections['detection_masks_reframed']

  min_score_thresh=scorethresh
  img = cv2.imread(image_path)
  nombre = counter
  plt.savefig(image_save+'/RESULTADOS_SR/'+nombre,  dpi=dpi ,bbox_inches='tight',pad_inches = 0)
  plt.clf()

  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']

  min_score_thresh = scorethresh
  salida = []
  salida.append(boxes.tolist())
  salida.append(scores[0])
  salida.append(clases_detected) 

  image = Image.open(image_path)
  width, height = image.size

  salida.append(0)
  salida.append(0)
  salida.append(mascaras_finales)

  con_propuesta = len(boxes.tolist())
  

  salida.append(0)
  salida.append(0)
  return salida


In [56]:
#%%capture
from pycocotools import mask
from skimage import measure

diccionario_data_gt = {
    'zurich':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/zurich_122_instancesonly_filtered_gtFine_val.json',
    'weimar':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/weimar_142_instancesonly_filtered_gtFine_val.json',
    'ulm':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/ulm_95_instancesonly_filtered_gtFine_val.json',
    'tubingen':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/tubingen_144_instancesonly_filtered_gtFine_val.json',
    'stuttgart':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/stuttgart_196_instancesonly_filtered_gtFine_val.json',
    'strasbourg':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/strasbourg_365_instancesonly_filtered_gtFine_val.json',
    'munster':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/munster_174_instancesonly_filtered_gtFine_val.json',
    'monchengladbach':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/monchengladbach_94_instancesonly_filtered_gtFine_val.json',
    'lindau':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/lindau_59_instancesonly_filtered_gtFine_val.json',
    'krefeld':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/krefeld_99_instancesonly_filtered_gtFine_val.json',
    'jena':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/jena_119_instancesonly_filtered_gtFine_val.json',
    'hanover':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/hanover_196_instancesonly_filtered_gtFine_val.json',
    'hamburg':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/hamburg_248_instancesonly_filtered_gtFine_val.json',
    'frankfurt':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/frankfurt_267_instancesonly_filtered_gtFine_val.json',
    'erfurt':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/erfurt_109_instancesonly_filtered_gtFine_val.json',
    'dusseldorf':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/dusseldorf_221_instancesonly_filtered_gtFine_val.json',
    'darmstadt':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/darmstadt_85_instancesonly_filtered_gtFine_val.json',
    'cologne':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/cologne_154_instancesonly_filtered_gtFine_val.json',
    'bremen':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/bremen_316_instancesonly_filtered_gtFine_val.json',
    'bochum':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/bochum_96_instancesonly_filtered_gtFine_val.json',
    'aachen':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/aachen_174_instancesonly_filtered_gtFine_val.json',
    'test':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/test.json'
}


datasets = os.listdir('/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/')

datasets = ['zurich','ulm','munster','monchengladbach','lindau','krefeld','jena','erfurt','darmstadt','bochum','frankfurt']
#datasets2 = ['weimar', 'tubingen', 'stuttgart', 'hanover', 'cologne', 'aachen' ,'dusseldorf' ,'hamburg' ,'bremen', 'strasbourg']
#datasets = ['lindau']
datasets = ['ulm','jena','munster','bochum']

for selected_dataset in datasets:
    
    if selected_dataset in diccionario_data_gt:


        jsonString_mod = diccionario_data_gt[selected_dataset]
        diccionario_ids_mod = {}

        
        with open(jsonString_mod) as json_file_mod:
            data_mod = json.load(json_file_mod)
            for p3_mod in data_mod['images']:                  
                data_s_image_mod = p3_mod['file_name'].replace('leftImg8bit/val/'+str(selected_dataset)+'/','')
                #data_s_image_mod = p3_mod['file_name'].replace('leftImg8bit/val/munster/','')
                data_s_id_mod = p3_mod['id']
                diccionario_ids_mod[data_s_image_mod] = data_s_id_mod

        #MASKRCNN-EXTRA-HOTFIX-
        MODELO_ELEGIDO = 'MASKRCNN-HOTFIX-DIRECTSR-NEWMASKOK-'+str(selected_dataset)

        imagenes_dir2 = '/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/'+str(selected_dataset)
        imagenes_dir3 = '/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/'+str(selected_dataset)

        contenido2 = os.listdir(imagenes_dir2)
        imagenes_dir2 = []
        counter = 0
        result = []
        result2 = []


        #PLEASE INSERT THE MINIMUN SCORE TO DETECT ONE OBJECT


        #PLEASE INSERT % OF THE WINDOWS SIZE R TO BE EVALLUATED
        WSizeR =  [0, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 1]
        #WSizeR = [1, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10, 0]
        WSizeR = [0]


        for window in WSizeR:


            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+'/')
            #create_dir('/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS_SR/')
            #create_dir('usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/AUX/')
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+"/SALIDA/")
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+"/AUX/")
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+"/AUX_COMPLEJO/")
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+"/JSON/")
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+"/TEMP/")
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+'/RESULTADOS_SR/')
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1//'+MODELO_ELEGIDO+'/'+str(window)+'/AUX/')
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1//'+MODELO_ELEGIDO+'/'+str(window)+'/FILES/')




            tiempos_procesamiento2 = []
            array_veces = []
            tiempos_procesar2 =  []

            out = []
            out2 = []
            concon = 0
            result2 = []
            out222 = []

            out5 = []


            '''RECORRO LAS IMÁGENES ALMACENADAS EN UN DIRECTORIO'''
            for fichero2 in contenido2:
              #x = re.search("[0-9]+[0-9]*", fichero2)
              #id = x.sgroup(0)
              image_path_gg = imagenes_dir3+"/"+str(fichero2)
              print(image_path_gg)
              
              contador_veces = 0
                
              createSR(image_path_gg)

              #salida = make_inference240(image_path_gg,fichero2,'/opt/share/IWINAC-EXTENDED-FOLDER/experimentos/'+MODELO_ELEGIDO+'/'+str(window)+'/',0,20,window,0.7,[1,2,3,4,8])
              salida = make_inference240RAW('/opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png',fichero2,'/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+'/',0,20,0.7,[1,2,3,4,8])


              print('he terminado')
              counter = counter+1

              result2 = []
              result3 = []
              out3 = []

              '''OBTENGO EL ÍNDICE PERTENECIENTE A DICHA IMAGEN'''
              #converted_num = int(id)
              converted_num = diccionario_ids_mod[fichero2]

              width = salida[3]
              height = salida[4]
              mascaras_finales = salida[5]
              boxes2 = []

              '''REALIZO LA TRANSOFORMACIÓN DE COORDENADAS DE ACUERDO CON EL FORMATO DE COCO'''
              contador_mascara = 0
              punterog = 0
              result = []
              result2 = []
              out39 = []
              result39 = []


              cluster_bueno = []
              numero_subimagenes = salida[7]
              numero_subimagenes_sin_propuesta = salida[6]

              '''
              for indexcluster in range(len(salida[5])):
                tripla = salida[5][indexcluster]

                lista_boxes_lista = tripla[0].tolist()
                lista_clases_lista = tripla[1].tolist()
                lista_scores_lista = tripla[2].tolist()
                cluster_aux = []
                cluster_aux.append(lista_boxes_lista)
                cluster_aux.append(lista_clases_lista)
                cluster_aux.append(lista_scores_lista)
                cluster_bueno.append(cluster_aux)

                "triplas": cluster_bueno,
              '''
              result2.extend(
                    [
                        {
                            "image_id": converted_num,
                            "numero_subimagenes": int(numero_subimagenes),
                            "numero_subimagenes_sin_propuesta":numero_subimagenes_sin_propuesta,
                        }
                    ]
              )


              for box in salida[0]:


                ymin = int(box[0]*height)
                xmin = int(box[1]*width)
                ymax = int(box[2]*height)
                xmax = int(box[3]*width) 

                box_new = []
                box_new.append(xmin)
                box_new.append(ymin)
                box_new.append(xmax-xmin)
                box_new.append(ymax-ymin)




                ground_truth_binary_mask = mascaras_finales[contador_mascara]
                contador_mascara = contador_mascara + 1
                
                

                fortran_ground_truth_binary_mask = np.asfortranarray(ground_truth_binary_mask)
                encoded_ground_truth = mask.encode(fortran_ground_truth_binary_mask)
                ground_truth_area = mask.area(encoded_ground_truth)
                ground_truth_bounding_box = mask.toBbox(encoded_ground_truth)
                contours = measure.find_contours(ground_truth_binary_mask, 0.5)



                annotation = {
                    "image_id": 1,
                    "category_id": 1,
                    "bbox": [],
                    "score": 1,
                    "segmentation": [],
                    "area": []
                }

                annotation["image_id"]= converted_num
                annotation["category_id"]= int(salida[2][punterog])
                #cambiar box_new por box para RAW
                annotation["bbox"]= ground_truth_bounding_box.tolist()
                annotation["score"]= salida[1][punterog].astype(float)
                annotation["area"]= ground_truth_area.tolist()
                punterog = punterog + 1

                for contour in contours:
                    contour = np.flip(contour, axis=1)
                    segmentation = contour.ravel().tolist()
                    annotation["segmentation"].append(segmentation)

                if len(annotation["segmentation"]) > 0:
                        result.extend([annotation])
                        result39.extend([annotation])


                '''print(annotation)'''
              '''
              for sample39 in result39:    
                out39.append(sample39)

              nombre_json = '/opt/share/IWINAC-EXTENDED-FOLDER/experimentos/'+MODELO_ELEGIDO+'/'+str(window)+'/JSON/'+str(converted_num)+'.json'
              with open(nombre_json, 'w', encoding='utf-8') as file39:
                json.dump(out39, file39, ensure_ascii=False)
              '''

              for sample in result:    
                out5.append(sample)

              for sample2 in result2:    
                out222.append(sample2)

              '''GUARDO EL JSON EN UN ARCHIVO'''
            with open('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+'/'+'test_data_modificado.json', 'w', encoding='utf-8') as file:
                json.dump(out5, file, ensure_ascii=False)

            with open('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+'/'+'test_data_modificado2.json', 'w', encoding='utf-8') as file2:
                json.dump(out222, file2, ensure_ascii=False)



/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000051_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000057_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000039_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000093_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000020_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png...

Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000029_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000030_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000004_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000060_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000073_000019_l

he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000017_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000009_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000046_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000045_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000034_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados

Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000066_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000006_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000091_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000105_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_00000

Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000074_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000103_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000036_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000093_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_00010

...

he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000106_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000103_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000002_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000011_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000004_000019_leftImg8bit.png
Running inference for /opt

he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000031_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000097_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000075_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000139_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000151_000019_leftImg8bit.png
Running inference for /opt

he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_007950_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_014658_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_033531_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_024855_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_009951_leftImg8bit.png
Running inference for /opt/share/IWI

Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_033714_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_028297_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_021479_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_020776_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA

he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_010562_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_027951_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_004229_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_023040_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/resultados/0D_SR.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_009554_leftImg8bit.png
Running inference for /opt/share/IWI

In [44]:
#%%capture
from pycocotools import mask
from skimage import measure

diccionario_data_gt = {
    'zurich':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/zurich_122_instancesonly_filtered_gtFine_val.json',
    'weimar':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/weimar_142_instancesonly_filtered_gtFine_val.json',
    'ulm':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/ulm_95_instancesonly_filtered_gtFine_val.json',
    'tubingen':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/tubingen_144_instancesonly_filtered_gtFine_val.json',
    'stuttgart':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/stuttgart_196_instancesonly_filtered_gtFine_val.json',
    'strasbourg':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/strasbourg_365_instancesonly_filtered_gtFine_val.json',
    'munster':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/munster_174_instancesonly_filtered_gtFine_val.json',
    'monchengladbach':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/monchengladbach_94_instancesonly_filtered_gtFine_val.json',
    'lindau':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/lindau_59_instancesonly_filtered_gtFine_val.json',
    'krefeld':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/krefeld_99_instancesonly_filtered_gtFine_val.json',
    'jena':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/jena_119_instancesonly_filtered_gtFine_val.json',
    'hanover':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/hanover_196_instancesonly_filtered_gtFine_val.json',
    'hamburg':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/hamburg_248_instancesonly_filtered_gtFine_val.json',
    'frankfurt':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/frankfurt_267_instancesonly_filtered_gtFine_val.json',
    'erfurt':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/erfurt_109_instancesonly_filtered_gtFine_val.json',
    'dusseldorf':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/dusseldorf_221_instancesonly_filtered_gtFine_val.json',
    'darmstadt':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/darmstadt_85_instancesonly_filtered_gtFine_val.json',
    'cologne':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/cologne_154_instancesonly_filtered_gtFine_val.json',
    'bremen':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/bremen_316_instancesonly_filtered_gtFine_val.json',
    'bochum':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/bochum_96_instancesonly_filtered_gtFine_val.json',
    'aachen':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/aachen_174_instancesonly_filtered_gtFine_val.json',
    'test':'/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLGT/test.json'
}


datasets = os.listdir('/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/')

datasets = ['zurich','ulm','munster','monchengladbach','lindau','krefeld','jena','erfurt','darmstadt','bochum','frankfurt']
#datasets2 = ['weimar', 'tubingen', 'stuttgart', 'hanover', 'cologne', 'aachen' ,'dusseldorf' ,'hamburg' ,'bremen', 'strasbourg']
#datasets = ['lindau']
datasets = ['ulm','jena','munster','bochum']

for selected_dataset in datasets:
    
    if selected_dataset in diccionario_data_gt:


        jsonString_mod = diccionario_data_gt[selected_dataset]
        diccionario_ids_mod = {}

        
        with open(jsonString_mod) as json_file_mod:
            data_mod = json.load(json_file_mod)
            for p3_mod in data_mod['images']:                  
                data_s_image_mod = p3_mod['file_name'].replace('leftImg8bit/val/'+str(selected_dataset)+'/','')
                #data_s_image_mod = p3_mod['file_name'].replace('leftImg8bit/val/munster/','')
                data_s_id_mod = p3_mod['id']
                diccionario_ids_mod[data_s_image_mod] = data_s_id_mod

        #MASKRCNN-EXTRA-HOTFIX-
        MODELO_ELEGIDO = 'MASKRCNN-HOTFIX-DIRECTSR-ORIGINAL-'+str(selected_dataset)

        imagenes_dir2 = '/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/'+str(selected_dataset)
        imagenes_dir3 = '/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/'+str(selected_dataset)

        contenido2 = os.listdir(imagenes_dir2)
        imagenes_dir2 = []
        counter = 0
        result = []
        result2 = []


        #PLEASE INSERT THE MINIMUN SCORE TO DETECT ONE OBJECT


        #PLEASE INSERT % OF THE WINDOWS SIZE R TO BE EVALLUATED
        WSizeR =  [0, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 1]
        #WSizeR = [1, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10, 0]
        WSizeR = [0]


        for window in WSizeR:


            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+'/')
            #create_dir('/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS_SR/')
            #create_dir('usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/AUX/')
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+"/SALIDA/")
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+"/AUX/")
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+"/AUX_COMPLEJO/")
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+"/JSON/")
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+"/TEMP/")
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+'/RESULTADOS_SR/')
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1//'+MODELO_ELEGIDO+'/'+str(window)+'/AUX/')
            create_dir('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1//'+MODELO_ELEGIDO+'/'+str(window)+'/FILES/')




            tiempos_procesamiento2 = []
            array_veces = []
            tiempos_procesar2 =  []

            out = []
            out2 = []
            concon = 0
            result2 = []
            out222 = []

            out5 = []


            '''RECORRO LAS IMÁGENES ALMACENADAS EN UN DIRECTORIO'''
            for fichero2 in contenido2:
              #x = re.search("[0-9]+[0-9]*", fichero2)
              #id = x.sgroup(0)
              image_path_gg = imagenes_dir3+"/"+str(fichero2)
              print(image_path_gg)
              
              contador_veces = 0
                
              

              #salida = make_inference240(image_path_gg,fichero2,'/opt/share/IWINAC-EXTENDED-FOLDER/experimentos/'+MODELO_ELEGIDO+'/'+str(window)+'/',0,20,window,0.7,[1,2,3,4,8])
              salida = make_inference240RAW(image_path_gg,fichero2,'/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+'/',0,20,0.7,[1,2,3,4,8])


              print('he terminado')
              counter = counter+1

              result2 = []
              result3 = []
              out3 = []

              '''OBTENGO EL ÍNDICE PERTENECIENTE A DICHA IMAGEN'''
              #converted_num = int(id)
              converted_num = diccionario_ids_mod[fichero2]

              width = salida[3]
              height = salida[4]
              mascaras_finales = salida[5]
              boxes2 = []

              '''REALIZO LA TRANSOFORMACIÓN DE COORDENADAS DE ACUERDO CON EL FORMATO DE COCO'''
              contador_mascara = 0
              punterog = 0
              result = []
              result2 = []
              out39 = []
              result39 = []


              cluster_bueno = []
              numero_subimagenes = salida[7]
              numero_subimagenes_sin_propuesta = salida[6]

              '''
              for indexcluster in range(len(salida[5])):
                tripla = salida[5][indexcluster]

                lista_boxes_lista = tripla[0].tolist()
                lista_clases_lista = tripla[1].tolist()
                lista_scores_lista = tripla[2].tolist()
                cluster_aux = []
                cluster_aux.append(lista_boxes_lista)
                cluster_aux.append(lista_clases_lista)
                cluster_aux.append(lista_scores_lista)
                cluster_bueno.append(cluster_aux)

                "triplas": cluster_bueno,
              '''
              result2.extend(
                    [
                        {
                            "image_id": converted_num,
                            "numero_subimagenes": int(numero_subimagenes),
                            "numero_subimagenes_sin_propuesta":numero_subimagenes_sin_propuesta,
                        }
                    ]
              )


              for box in salida[0]:


                ymin = int(box[0]*height)
                xmin = int(box[1]*width)
                ymax = int(box[2]*height)
                xmax = int(box[3]*width) 

                box_new = []
                box_new.append(xmin)
                box_new.append(ymin)
                box_new.append(xmax-xmin)
                box_new.append(ymax-ymin)




                ground_truth_binary_mask = mascaras_finales[contador_mascara]
                contador_mascara = contador_mascara + 1
                
                

                fortran_ground_truth_binary_mask = np.asfortranarray(ground_truth_binary_mask)
                encoded_ground_truth = mask.encode(fortran_ground_truth_binary_mask)
                ground_truth_area = mask.area(encoded_ground_truth)
                ground_truth_bounding_box = mask.toBbox(encoded_ground_truth)
                contours = measure.find_contours(ground_truth_binary_mask, 0.5)



                annotation = {
                    "image_id": 1,
                    "category_id": 1,
                    "bbox": [],
                    "score": 1,
                    "segmentation": [],
                    "area": []
                }

                annotation["image_id"]= converted_num
                annotation["category_id"]= int(salida[2][punterog])
                #cambiar box_new por box para RAW
                annotation["bbox"]= ground_truth_bounding_box.tolist()
                annotation["score"]= salida[1][punterog].astype(float)
                annotation["area"]= ground_truth_area.tolist()
                punterog = punterog + 1

                for contour in contours:
                    contour = np.flip(contour, axis=1)
                    segmentation = contour.ravel().tolist()
                    annotation["segmentation"].append(segmentation)

                if len(annotation["segmentation"]) > 0:
                        result.extend([annotation])
                        result39.extend([annotation])


                '''print(annotation)'''
              '''
              for sample39 in result39:    
                out39.append(sample39)

              nombre_json = '/opt/share/IWINAC-EXTENDED-FOLDER/experimentos/'+MODELO_ELEGIDO+'/'+str(window)+'/JSON/'+str(converted_num)+'.json'
              with open(nombre_json, 'w', encoding='utf-8') as file39:
                json.dump(out39, file39, ensure_ascii=False)
              '''

              for sample in result:    
                out5.append(sample)

              for sample2 in result2:    
                out222.append(sample2)

              '''GUARDO EL JSON EN UN ARCHIVO'''
            with open('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+'/'+'test_data_modificado.json', 'w', encoding='utf-8') as file:
                json.dump(out5, file, ensure_ascii=False)

            with open('/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/'+MODELO_ELEGIDO+'/'+str(window)+'/'+'test_data_modificado2.json', 'w', encoding='utf-8') as file2:
                json.dump(out222, file2, ensure_ascii=False)



/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000051_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000051_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000057_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000057_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000039_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000039_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000093_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000093_000019_leftI

he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000028_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000028_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000079_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000079_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000059_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000059_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000018_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000018

he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000050_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000050_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000074_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000074_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000010_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000010_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000062_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/ulm/ulm_000062

he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000109_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000109_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000068_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000068_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000076_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000076_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000095_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/

he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000034_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000034_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000037_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000037_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000046_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000046_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/jena/jena_000089_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/

...

he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000109_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000109_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000116_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000116_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000005_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000005_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000062_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FO

/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000096_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000096_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000156_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000156_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000008_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000008_000019_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/munster/munster_000091_000019_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasI

he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_006746_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_006746_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_037223_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_037223_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_024343_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_024343_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_023648_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIW

he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_033714_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_033714_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_028297_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_028297_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_021479_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_021479_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_020776_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIW

he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_013705_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_013705_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_015038_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_015038_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_021606_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_021606_leftImg8bit.png... 
he terminado
/opt/share/IWINAC-EXTENDED-FOLDER/medidasIWINAC/FULLEXTENSION/FULLDATA/bochum/bochum_000000_004032_leftImg8bit.png
Running inference for /opt/share/IWINAC-EXTENDED-FOLDER/medidasIW

In [46]:
print(diccionario_ids_mod['bochum_000000_004229_leftImg8bit.png'])

90


In [53]:
import json

a1 = '/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/MASKRCNN-HOTFIX-DIRECTSR-ORIGINAL-bochum/0/test_data_modificado.json'
a2 = '/opt/share/IWINAC-EXTENDED-FOLDER/experimentosrev1/MASKRCNN-HOTFIX-DIRECTSR-NEW-bochum/0/test_data_modificado.json'

f = open(a1)
data = json.load(f)

for i in data:
    if i['image_id'] == 90:
        print(i['bbox'])
        
print('*****')
        
f = open(a2)
data = json.load(f)

for i in data:
    if i['image_id'] == 90:
        print(i['bbox'])
        

[1139.0, 379.0, 560.0, 209.0]
[0.0, 377.0, 245.0, 235.0]
*****
[0.0, 750.0, 489.0, 476.0]
[2280.0, 759.0, 1118.0, 418.0]
